In [ ]:
import numpy as np
import scipy as sp
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
from random import sample
import csv
from func import *

# Load county information

In [ ]:
path = '../data/facebook-counties/'

In [ ]:
populations = []
state = []
with open(path+'facebook-county-friendship-metadata.csv', newline='') as csvfile:
    meta_data = csv.reader(csvfile, delimiter='\t', quotechar='|')
    for row in meta_data:
        populations.append(int(row[0]))
        state.append(row[2])

labels = []
with open(path+'facebook-county-friendship.labels', newline='') as csvfile:
    meta_data = csv.reader(csvfile, delimiter='\t', quotechar='|')
    for row in meta_data:
        labels.append(row[0])
        
fips = []
with open(path+'county-info.csv', newline='') as csvfile:
    meta_data = csv.reader(csvfile, delimiter=',', quotechar='|')
    next(meta_data)
    for row in meta_data:
        fips.append(row[0])
        
xy = np.loadtxt('../data/facebook-counties/facebook-county-friendship.xy')

# Read graph
#### We are going to remove the following nodes:
- counties in Hawaii and Alaska
- six disconnected counties in Virginia
- irregular node representing Loving, Texas

In [ ]:
G = nx.read_graphml('../data/facebook-counties/facebook-county.graphml', node_type=int)

removed = []
disconnected = ["Covington, VA", "Emporia, VA", "Fairfax, VA", 
                "Lexington, VA", "Manassas Park, VA", "Martinsville, VA"]
irregular = ["Loving, TX"]
for i in range(G.number_of_nodes()):
    if state[i] == 'HI' or state[i] == 'AK':
        removed.append(i+1)
    elif labels[i] in disconnected:
        print(labels[i])
        removed.append(i+1)
    elif labels[i] in irregular:
        print(labels[i])
        removed.append(i+1)
        
G.remove_nodes_from(removed)

#### Relabel nodes from 1 to n

In [ ]:
n = G.number_of_nodes()
mapping = dict(zip(G, range(1, n+1)))
G = nx.relabel_nodes(G, mapping)
map_to_original = dict((val,key-1) for key,val in mapping.items())

#### Remove edges that are longer than 500 miles

In [ ]:
from geopy.distance import distance

In [ ]:
ct = 0
for u,v in G.edges():
    if distance(xy[map_to_original[u]][::-1], xy[map_to_original[v]][::-1]).miles > 500:
        G.remove_edge(u,v)
        ct += 1

print(f'Number of edges removed: {ct}')
print(f'Number of edges in total: {G.number_of_edges()}')

#### Plot the graph

In [ ]:
pos = dict((i, xy[map_to_original[i],:]) for i in range(1,n+1))
nx.draw(G, pos, node_size=0, width=.1, alpha=1)

# Compute betweenness measures

In [ ]:
shortest_path_betweenness = nx.edge_betweenness_centrality(G)

In [ ]:
current_flow_betweenness = nx.edge_current_flow_betweenness_centrality(G)

In [ ]:
eigenvector_centrality = nx.eigenvector_centrality(G, max_iter=1000)

In [ ]:
from julia.api import Julia
jl = Julia(compiled_modules=False)
from julia import Main

In [ ]:
Main.include("../local_flow_betweenness.jl");

In [ ]:
local50_betweenness = Main.local_flow_betweenness(np.array(G.nodes()), list(G.edges()), locality_index=.5)
local10_betweenness = Main.local_flow_betweenness(np.array(G.nodes()), list(G.edges()), locality_index=.1)
local02_betweenness = Main.local_flow_betweenness(np.array(G.nodes()), list(G.edges()), locality_index=.02)

# Intervention and simulation results

In [ ]:
Main.include("../utils.jl");
Main.include("../networkSEIR.jl");

In [ ]:
total_population = 0
for i in G.nodes():
    total_population += populations[map_to_original[i]]

In [ ]:
A = (nx.adjacency_matrix(G) + sp.sparse.eye(G.number_of_nodes())).tocsc()

## Scenario 1a: 85% final size, random initialization

In [ ]:
beta=0.0315

#### Verify that $\beta$ is set appropriately

In [ ]:
# initial conditions
S_0 = np.zeros(n)
E_0 = np.zeros(n)
I_0 = np.zeros(n)
R_0 = np.zeros(n)
for i in range(n):
    S_0[i] = populations[map_to_original[i+1]]

# the following lines set up random initialization
for i in sample(range(n),31):
    E_0[i] = 0
    I_0[i] = populations[map_to_original[i+1]] * .001
    S_0[i] = S_0[i] - E_0[i] - I_0[i]

# the following lines set up cluster initialization
"""
for i in initial_cluster:
    E_0[i] = 0
    I_0[i] = populations[map_to_original[i+1]] * .001
    S_0[i] = S_0[i] - E_0[i] - I_0[i]
"""

# the following lines set up NY initialization
"""
for i in range(n):
    county_no = map_to_original[i+1]
    if labels[county_no] == 'New York, NY':
        E_0[i] = 0
        I_0[i] = populations[county_no] * .001
        S_0[i] = S_0[i] - E_0[i] - I_0[i]
"""

# the following lines set up LA initialization
"""
for i in range(n):
    county_no = map_to_original[i+1]
    if labels[county_no] == 'Los Angeles, CA':
        E_0[i] = 0
        I_0[i] = populations[county_no] * .001
        S_0[i] = S_0[i] - E_0[i] - I_0[i]
"""

# the following lines set up Chicago initialization
"""
for i in range(n):
    county_no = map_to_original[i+1]
    if labels[county_no] == 'Cook, IL':
        E_0[i] = 0
        I_0[i] = populations[county_no] * .001
        S_0[i] = S_0[i] - E_0[i] - I_0[i]
"""
    
ini_cond = np.column_stack((S_0,E_0,I_0,R_0))

# time span
t_end = 400.
t_span = (0.,t_end)

sol = Main.run_network_seir(Main.scipyCSC_to_julia(A), ini_cond, t_span, beta=beta)
print(f'This should be around 0.85: {get_total_active_cases(sol)/total_population:.6f}')

In [ ]:
sol = Main.run_network_seir(Main.scipyCSC_to_julia(A), ini_cond, t_span, beta=beta)
sum_s_ni, sum_e_ni, sum_i_ni, sum_r_ni = get_data_for_plotting(sol)
t_int = np.linspace(0, int(t_end), num=int(t_end)+1)
plt.plot(t_int,(sum_e_ni+sum_i_ni)/total_population, label='NI', linestyle='-', color='dimgray', linewidth=3, alpha=1)

##### Sample epidemic curve: random initialization, 25% intervention, no delay

In [ ]:
perc = .25
reduced_weight = .1

In [ ]:
A_ui = ((1-(1-reduced_weight)*perc)*nx.adjacency_matrix(G) + sp.sparse.eye(G.number_of_nodes())).tocsc()
A_hd = create_weighted_adjacency_from_degree_dist(G, perc, weight=reduced_weight)
A_eg = create_weighted_adjacency_from_node_betweenness(G, eigenvector_centrality, perc, weight=reduced_weight)
A_sp = create_weighted_adjacency_from_edge_betweenness(G, shortest_path_betweenness, perc, weight=reduced_weight)
A_rw = create_weighted_adjacency_from_edge_betweenness(G, current_flow_betweenness, perc, weight=reduced_weight)
A_lf50 = create_weighted_adjacency_from_edge_betweenness(G, local50_betweenness, perc, weight=reduced_weight)
A_lf10 = create_weighted_adjacency_from_edge_betweenness(G, local10_betweenness, perc, weight=reduced_weight)
A_lf02 = create_weighted_adjacency_from_edge_betweenness(G, local02_betweenness, perc, weight=reduced_weight)

In [ ]:
# time span
t_end = 200.
t_span = (0., t_end)

# initial conditions
n = G.number_of_nodes()
S_0 = np.zeros(n)
E_0 = np.zeros(n)
I_0 = np.zeros(n)
R_0 = np.zeros(n)

for i in range(n):
    S_0[i] = populations[map_to_original[i+1]]

# the following lines set up random initialization
for i in sample(range(n),31):
    E_0[i] = 0
    I_0[i] = populations[map_to_original[i+1]] * .001
    S_0[i] = S_0[i] - E_0[i] - I_0[i]

ini_cond = np.column_stack((S_0,E_0,I_0,R_0))

# simulation
sol_ni = Main.run_network_seir(Main.scipyCSC_to_julia(A), ini_cond, t_span, beta=beta)
sol_ui = Main.run_network_seir(Main.scipyCSC_to_julia(A_ui), ini_cond, t_span, beta=beta)
sol_hd = Main.run_network_seir(Main.scipyCSC_to_julia(A_hd), ini_cond, t_span, beta=beta)
sol_eg = Main.run_network_seir(Main.scipyCSC_to_julia(A_eg), ini_cond, t_span, beta=beta)
sol_sp = Main.run_network_seir(Main.scipyCSC_to_julia(A_sp), ini_cond, t_span, beta=beta)
sol_rw = Main.run_network_seir(Main.scipyCSC_to_julia(A_rw), ini_cond, t_span, beta=beta)
sol_lf50 = Main.run_network_seir(Main.scipyCSC_to_julia(A_lf50), ini_cond, t_span, beta=beta)
sol_lf10 = Main.run_network_seir(Main.scipyCSC_to_julia(A_lf10), ini_cond, t_span, beta=beta)
sol_lf02 = Main.run_network_seir(Main.scipyCSC_to_julia(A_lf02), ini_cond, t_span, beta=beta)

# get curves and make plot
sum_s_ni, sum_e_ni, sum_i_ni, sum_r_ni = get_data_for_plotting(sol_ni)
sum_s_ui, sum_e_ui, sum_i_ui, sum_r_ui = get_data_for_plotting(sol_ui)
sum_s_hd, sum_e_hd, sum_i_hd, sum_r_hd = get_data_for_plotting(sol_hd)
sum_s_eg, sum_e_eg, sum_i_eg, sum_r_eg = get_data_for_plotting(sol_eg)
sum_s_sp, sum_e_sp, sum_i_sp, sum_r_sp = get_data_for_plotting(sol_sp)
sum_s_rw, sum_e_rw, sum_i_rw, sum_r_rw = get_data_for_plotting(sol_rw)
sum_s_lf50, sum_e_lf50, sum_i_lf50, sum_r_lf50 = get_data_for_plotting(sol_lf50)
sum_s_lf10, sum_e_lf10, sum_i_lf10, sum_r_lf10 = get_data_for_plotting(sol_lf10)
sum_s_lf02, sum_e_lf02, sum_i_lf02, sum_r_lf02 = get_data_for_plotting(sol_lf02)

t_int = np.linspace(0, int(t_end), num=int(t_end)+1)

plt.figure(figsize=(9.3,4.5))
plt.plot(t_int,(sum_e_ni+sum_i_ni)/total_population, label='NI', linestyle='-', color='dimgray', linewidth=3, alpha=1)
plt.plot(t_int,(sum_e_ui+sum_i_ui)/total_population, label='UI', linestyle=(0,(5,5)), color='k', linewidth=4, alpha=1)
plt.plot(t_int,(sum_e_eg+sum_i_eg)/total_population, label='EG', linestyle=(0,(3,1,1,1)), color='tab:brown', linewidth=4, alpha=1)
plt.plot(t_int,(sum_e_hd+sum_i_hd)/total_population, label='HD', linestyle=(0,(3,5,1,5)), color='tab:red', linewidth=4, alpha=1)
plt.plot(t_int,(sum_e_sp+sum_i_sp)/total_population, label='SP', linestyle=(0,(5,1)), color='tab:orange', linewidth=4, alpha=1)
plt.plot(t_int,(sum_e_rw+sum_i_rw)/total_population, label='CF', linestyle='dashed', color='tab:green', linewidth=4, alpha=1)
plt.plot(t_int,(sum_e_lf50+sum_i_lf50)/total_population, label='LF(1/2)', linestyle=(0,(1,1)), color="tab:cyan", linewidth=4, alpha=1)
plt.plot(t_int,(sum_e_lf10+sum_i_lf10)/total_population, label='LF(1/10)', linestyle=(0,(3,1,1,1,1,1)), color='tab:purple', linewidth=4, alpha=1)
plt.plot(t_int,(sum_e_lf02+sum_i_lf02)/total_population, label='LF(1/50)', linestyle='dashdot', color='tab:blue', linewidth=4)
leg = plt.legend(fontsize=18, bbox_to_anchor=(.5, 1.4), ncol=3, loc='upper center', handlelength=4)
for i in leg.legendHandles:
    i.set_linewidth(4)
plt.xticks(size=18)
plt.yticks((.0, .1), size=18)
plt.xlabel('Day', fontsize=22)
plt.ylabel('Active Cases',fontsize=22)
plt.savefig("facebook_curves_randinit_85.tiff", bbox_inches='tight', format='tiff', dpi=400, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

### Simulation for interventions that start on day 0, random initialization

In [ ]:
reduced_weight = .1
target_perc = [.05, .1, .15, .2, .25, .3, .35, .4, .45, .5]
num_scenarios = len(target_perc)
num_trials = 50

maxCases_ni = np.zeros((num_trials,num_scenarios))
maxCases_ui = np.zeros((num_trials,num_scenarios))
maxCases_hd = np.zeros((num_trials,num_scenarios))
maxCases_eg = np.zeros((num_trials,num_scenarios))
maxCases_sp = np.zeros((num_trials,num_scenarios))
maxCases_rw = np.zeros((num_trials,num_scenarios))
maxCases_lf50 = np.zeros((num_trials,num_scenarios))
maxCases_lf10 = np.zeros((num_trials,num_scenarios))
maxCases_lf02 = np.zeros((num_trials,num_scenarios))
totalCases_ni = np.zeros((num_trials,num_scenarios))
totalCases_ui = np.zeros((num_trials,num_scenarios))
totalCases_hd = np.zeros((num_trials,num_scenarios))
totalCases_eg = np.zeros((num_trials,num_scenarios))
totalCases_sp = np.zeros((num_trials,num_scenarios))
totalCases_rw = np.zeros((num_trials,num_scenarios))
totalCases_lf50 = np.zeros((num_trials,num_scenarios))
totalCases_lf10 = np.zeros((num_trials,num_scenarios))
totalCases_lf02 = np.zeros((num_trials,num_scenarios))

for k in range(num_scenarios):
    
    perc = target_perc[k]
    A_ui = ((1-(1-reduced_weight)*perc)*nx.adjacency_matrix(G) + sp.sparse.eye(G.number_of_nodes())).tocsc()
    A_hd = create_weighted_adjacency_from_degree_dist(G, perc, weight=reduced_weight)
    A_eg = create_weighted_adjacency_from_node_betweenness(G, eigenvector_centrality, perc, weight=reduced_weight)
    A_sp = create_weighted_adjacency_from_edge_betweenness(G, shortest_path_betweenness, perc, weight=reduced_weight)
    A_rw = create_weighted_adjacency_from_edge_betweenness(G, current_flow_betweenness, perc, weight=reduced_weight)
    A_lf50 = create_weighted_adjacency_from_edge_betweenness(G, local50_betweenness, perc, weight=reduced_weight)
    A_lf10 = create_weighted_adjacency_from_edge_betweenness(G, local10_betweenness, perc, weight=reduced_weight)
    A_lf02 = create_weighted_adjacency_from_edge_betweenness(G, local02_betweenness, perc, weight=reduced_weight)
    
    for trial in range(num_trials):
        
        print(f'perc = {perc:.2f}, trial {trial+1:d} of {num_trials:d}', end="\n")
    
        S_0 = np.zeros(n)
        E_0 = np.zeros(n)
        I_0 = np.zeros(n)
        R_0 = np.zeros(n)
        for i in range(n):
            S_0[i] = populations[map_to_original[i+1]]

        for i in sample(range(n),31):
            E_0[i] = 0
            I_0[i] = populations[map_to_original[i+1]] * .001
            S_0[i] = S_0[i] - E_0[i] - I_0[i]

        ini_cond = np.column_stack((S_0,E_0,I_0,R_0))
        t_span = (0.,1000.)
        
        sol_ni = Main.run_network_seir(Main.scipyCSC_to_julia(A), ini_cond, t_span, beta=beta)
        sol_ui = Main.run_network_seir(Main.scipyCSC_to_julia(A_ui), ini_cond, t_span, beta=beta)
        sol_hd = Main.run_network_seir(Main.scipyCSC_to_julia(A_hd), ini_cond, t_span, beta=beta)
        sol_eg = Main.run_network_seir(Main.scipyCSC_to_julia(A_eg), ini_cond, t_span, beta=beta)
        sol_sp = Main.run_network_seir(Main.scipyCSC_to_julia(A_sp), ini_cond, t_span, beta=beta)
        sol_rw = Main.run_network_seir(Main.scipyCSC_to_julia(A_rw), ini_cond, t_span, beta=beta)
        sol_lf50 = Main.run_network_seir(Main.scipyCSC_to_julia(A_lf50), ini_cond, t_span, beta=beta)
        sol_lf10 = Main.run_network_seir(Main.scipyCSC_to_julia(A_lf10), ini_cond, t_span, beta=beta)
        sol_lf02 = Main.run_network_seir(Main.scipyCSC_to_julia(A_lf02), ini_cond, t_span, beta=beta) 
        maxCases_ni[trial,k] = get_max_active_cases(sol_ni)
        maxCases_ui[trial,k] = get_max_active_cases(sol_ui)
        maxCases_hd[trial,k] = get_max_active_cases(sol_hd)
        maxCases_eg[trial,k] = get_max_active_cases(sol_eg)
        maxCases_sp[trial,k] = get_max_active_cases(sol_sp)
        maxCases_rw[trial,k] = get_max_active_cases(sol_rw)
        maxCases_lf50[trial,k] = get_max_active_cases(sol_lf50)
        maxCases_lf10[trial,k] = get_max_active_cases(sol_lf10)
        maxCases_lf02[trial,k] = get_max_active_cases(sol_lf02)
        totalCases_ni[trial,k] = get_total_active_cases(sol_ni)
        totalCases_ui[trial,k] = get_total_active_cases(sol_ui)
        totalCases_hd[trial,k] = get_total_active_cases(sol_hd)
        totalCases_eg[trial,k] = get_total_active_cases(sol_eg)
        totalCases_sp[trial,k] = get_total_active_cases(sol_sp)
        totalCases_rw[trial,k] = get_total_active_cases(sol_rw)
        totalCases_lf50[trial,k] = get_total_active_cases(sol_lf50)
        totalCases_lf10[trial,k] = get_total_active_cases(sol_lf10)
        totalCases_lf02[trial,k] = get_total_active_cases(sol_lf02)

In [ ]:
maxCases_ni_mean = np.mean(maxCases_ni/total_population, axis=0)
maxCases_ui_mean = np.mean(maxCases_ui/total_population, axis=0)
maxCases_hd_mean = np.mean(maxCases_hd/total_population, axis=0)
maxCases_eg_mean = np.mean(maxCases_eg/total_population, axis=0)
maxCases_sp_mean = np.mean(maxCases_sp/total_population, axis=0)
maxCases_rw_mean = np.mean(maxCases_rw/total_population, axis=0)
maxCases_lf50_mean = np.mean(maxCases_lf50/total_population, axis=0)
maxCases_lf10_mean = np.mean(maxCases_lf10/total_population, axis=0)
maxCases_lf02_mean = np.mean(maxCases_lf02/total_population, axis=0)
totalCases_ni_mean = np.mean(totalCases_ni/total_population, axis=0)
totalCases_ui_mean = np.mean(totalCases_ui/total_population, axis=0)
totalCases_hd_mean = np.mean(totalCases_hd/total_population, axis=0)
totalCases_eg_mean = np.mean(totalCases_eg/total_population, axis=0)
totalCases_sp_mean = np.mean(totalCases_sp/total_population, axis=0)
totalCases_rw_mean = np.mean(totalCases_rw/total_population, axis=0)
totalCases_lf50_mean = np.mean(totalCases_lf50/total_population, axis=0)
totalCases_lf10_mean = np.mean(totalCases_lf10/total_population, axis=0)
totalCases_lf02_mean = np.mean(totalCases_lf02/total_population, axis=0)

plt.figure(figsize=(9.3,4.5))
plt.plot(target_perc, maxCases_ni_mean, label='NI', linestyle='-', color='dimgray', linewidth=6)
plt.plot(target_perc, maxCases_ui_mean, label='UI', linestyle=(0,(5,5)), color='k', linewidth=6)
plt.plot(target_perc, maxCases_hd_mean, label='HD', linestyle=(0,(3,5,1,5)), color='tab:red', linewidth=6)
plt.plot(target_perc, maxCases_eg_mean, label='EG', linestyle=(0,(3,1,1,1)), color='tab:brown', linewidth=6)
plt.plot(target_perc, maxCases_sp_mean, label='SP', linestyle=(0,(5,1)), color='tab:orange', linewidth=6)
plt.plot(target_perc, maxCases_rw_mean, label='CF', linestyle='dashed', color='tab:green', linewidth=6)
plt.plot(target_perc, maxCases_lf50_mean, label='LF(1/2)', linestyle=(0,(1,1)), color="tab:cyan", linewidth=6)
plt.plot(target_perc, maxCases_lf10_mean, label='LF(1/10)', linestyle=(0,(3,1,1,1,1,1)), color='tab:purple', linewidth=6)
plt.plot(target_perc, maxCases_lf02_mean, label='LF(1/50)', linestyle='dashdot', color='tab:blue', linewidth=6)
leg = plt.legend(fontsize=18, bbox_to_anchor=(.5, 1.4), ncol=3, loc='upper center', handlelength=4)
for i in leg.legendHandles:
    i.set_linewidth(4)
plt.xticks((.1, .2, .3, .4, .5), ('$10\%$', '$20\%$', '$30\%$', '$40\%$', '$50\%$'), color='k', size=18)
plt.yticks((.1, .2), size=18)
plt.xlabel('Percentage of Targeted Edges', fontsize=22)
plt.ylabel('Epidemic Peak',fontsize=22)
plt.savefig("facebook_epipeak_randinit_85.tiff", bbox_inches='tight', format='tiff', dpi=400, pil_kwargs={"compression": "tiff_lzw"})
plt.show()
                                      
plt.figure(figsize=(9.3,4.5))
plt.plot(target_perc, totalCases_ni_mean, label='NI', linestyle='-', color='dimgray', linewidth=6)
plt.plot(target_perc, totalCases_ui_mean, label='UI', linestyle=(0,(5,5)), color='k', linewidth=6)
plt.plot(target_perc, totalCases_hd_mean, label='HD', linestyle=(0,(3,5,1,5)), color='tab:red', linewidth=6)
plt.plot(target_perc, totalCases_eg_mean, label='EG', linestyle=(0,(3,1,1,1)), color='tab:brown', linewidth=6)
plt.plot(target_perc, totalCases_sp_mean, label='SP', linestyle=(0,(5,1)), color='tab:orange', linewidth=6)
plt.plot(target_perc, totalCases_rw_mean, label='CF', linestyle='dashed', color='tab:green', linewidth=6)
plt.plot(target_perc, totalCases_lf50_mean, label='LF(1/2)', linestyle=(0,(1,1)), color="tab:cyan", linewidth=6)
plt.plot(target_perc, totalCases_lf10_mean, label='LF(1/10)', linestyle=(0,(3,1,1,1,1,1)), color='tab:purple', linewidth=6)
plt.plot(target_perc, totalCases_lf02_mean, label='LF(1/50)', linestyle='dashdot', color='tab:blue', linewidth=6)
leg = plt.legend(fontsize=18, bbox_to_anchor=(.5, 1.4), ncol=3, loc='upper center', handlelength=4)
for i in leg.legendHandles:
    i.set_linewidth(4)
plt.xticks((.1, .2, .3, .4, .5), ('$10\%$', '$20\%$', '$30\%$', '$40\%$', '$50\%$'), color='k', size=18)
plt.yticks(size=18)
plt.xlabel('Percentage of Targeted Edges', fontsize=22)
plt.ylabel('Final Epidemic Size',fontsize=22)
plt.savefig("facebook_episize_randinit_85.tiff", bbox_inches='tight', format='tiff', dpi=400, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

## Scenario 1b: 85% final size, cluster initialization

In [ ]:
initial_states = ['NH', 'ME', 'MA', 'CT', 'VT', 'RI']
initial_cluster = []
for i in G.nodes():
    if state[map_to_original[i]] in initial_states:
        initial_cluster.append(i)

In [ ]:
beta=0.0315

#### Verify that $\beta$ is set appropriately

In [ ]:
# initial conditions
S_0 = np.zeros(n)
E_0 = np.zeros(n)
I_0 = np.zeros(n)
R_0 = np.zeros(n)
for i in range(n):
    S_0[i] = populations[map_to_original[i+1]]  

# the following lines set up cluster initialization
for i in initial_cluster:
    E_0[i] = 0
    I_0[i] = populations[map_to_original[i+1]] * .001
    S_0[i] = S_0[i] - E_0[i] - I_0[i]
    
ini_cond = np.column_stack((S_0,E_0,I_0,R_0))

# time span
t_end = 400.
t_span = (0.,t_end)

sol = Main.run_network_seir(Main.scipyCSC_to_julia(A), ini_cond, t_span, beta=beta)
print(f'This should be around 0.85: {get_total_active_cases(sol)/total_population:.6f}')

In [ ]:
# Save data for plotting
sum_S, sum_E, sum_I, sum_R = get_data_for_plotting(sol)
with open("facebookcounty_curves.txt", "w") as f:
    for i in range(len(sum_S)):
        f.write(f"{sum_S[i]/total_population:.6f}\t{sum_E[i]/total_population:.6f}\t{sum_I[i]/total_population:.6f}\t{sum_R[i]/total_population:.6f}\n")

In [ ]:
sol = Main.run_network_seir(Main.scipyCSC_to_julia(A), ini_cond, t_span, beta=beta)
sum_s_ni, sum_e_ni, sum_i_ni, sum_r_ni = get_data_for_plotting(sol)
t_int = np.linspace(0, int(t_end), num=int(t_end)+1)
plt.plot(t_int,(sum_e_ni+sum_i_ni)/total_population, label='NI', linestyle='-', color='dimgray', linewidth=3, alpha=1)

##### Sample epidemic curve: cluster initialization, no delay, 25% intervention

In [ ]:
perc = .25
reduced_weight = .1

In [ ]:
A_ui = ((1-(1-reduced_weight)*perc)*nx.adjacency_matrix(G) + sp.sparse.eye(G.number_of_nodes())).tocsc()
A_hd = create_weighted_adjacency_from_degree_dist(G, perc, weight=reduced_weight)
A_eg = create_weighted_adjacency_from_node_betweenness(G, eigenvector_centrality, perc, weight=reduced_weight)
A_sp = create_weighted_adjacency_from_edge_betweenness(G, shortest_path_betweenness, perc, weight=reduced_weight)
A_rw = create_weighted_adjacency_from_edge_betweenness(G, current_flow_betweenness, perc, weight=reduced_weight)
A_lf50 = create_weighted_adjacency_from_edge_betweenness(G, local50_betweenness, perc, weight=reduced_weight)
A_lf10 = create_weighted_adjacency_from_edge_betweenness(G, local10_betweenness, perc, weight=reduced_weight)
A_lf02 = create_weighted_adjacency_from_edge_betweenness(G, local02_betweenness, perc, weight=reduced_weight)

#### Predicted epidemic curve: 

In [ ]:
# time span
t_end = 400.
t_span = (0., t_end)

# initial conditions
n = G.number_of_nodes()
S_0 = np.zeros(n)
E_0 = np.zeros(n)
I_0 = np.zeros(n)
R_0 = np.zeros(n)

for i in range(n):
    S_0[i] = populations[map_to_original[i+1]]
    
# the following lines set up cluster initialization
for i in initial_cluster:
    E_0[i] = 0
    I_0[i] = populations[map_to_original[i+1]] * .001
    S_0[i] = S_0[i] - E_0[i] - I_0[i]

ini_cond = np.column_stack((S_0,E_0,I_0,R_0))

# simulation
sol_ni = Main.run_network_seir(Main.scipyCSC_to_julia(A), ini_cond, t_span, beta=beta)
sol_ui = Main.run_network_seir(Main.scipyCSC_to_julia(A_ui), ini_cond, t_span, beta=beta)
sol_hd = Main.run_network_seir(Main.scipyCSC_to_julia(A_hd), ini_cond, t_span, beta=beta)
sol_eg = Main.run_network_seir(Main.scipyCSC_to_julia(A_eg), ini_cond, t_span, beta=beta)
sol_sp = Main.run_network_seir(Main.scipyCSC_to_julia(A_sp), ini_cond, t_span, beta=beta)
sol_rw = Main.run_network_seir(Main.scipyCSC_to_julia(A_rw), ini_cond, t_span, beta=beta)
sol_lf50 = Main.run_network_seir(Main.scipyCSC_to_julia(A_lf50), ini_cond, t_span, beta=beta)
sol_lf10 = Main.run_network_seir(Main.scipyCSC_to_julia(A_lf10), ini_cond, t_span, beta=beta)
sol_lf02 = Main.run_network_seir(Main.scipyCSC_to_julia(A_lf02), ini_cond, t_span, beta=beta)

# get curves and make plot
sum_s_ni, sum_e_ni, sum_i_ni, sum_r_ni = get_data_for_plotting(sol_ni)
sum_s_ui, sum_e_ui, sum_i_ui, sum_r_ui = get_data_for_plotting(sol_ui)
sum_s_hd, sum_e_hd, sum_i_hd, sum_r_hd = get_data_for_plotting(sol_hd)
sum_s_eg, sum_e_eg, sum_i_eg, sum_r_eg = get_data_for_plotting(sol_eg)
sum_s_sp, sum_e_sp, sum_i_sp, sum_r_sp = get_data_for_plotting(sol_sp)
sum_s_rw, sum_e_rw, sum_i_rw, sum_r_rw = get_data_for_plotting(sol_rw)
sum_s_lf50, sum_e_lf50, sum_i_lf50, sum_r_lf50 = get_data_for_plotting(sol_lf50)
sum_s_lf10, sum_e_lf10, sum_i_lf10, sum_r_lf10 = get_data_for_plotting(sol_lf10)
sum_s_lf02, sum_e_lf02, sum_i_lf02, sum_r_lf02 = get_data_for_plotting(sol_lf02)

t_int = np.linspace(0, int(t_end), num=int(t_end)+1)

plt.figure(figsize=(9.3,4.5))
plt.plot(t_int,(sum_e_ni+sum_i_ni)/total_population, label='NI', linestyle='-', color='dimgray', linewidth=3, alpha=1)
plt.plot(t_int,(sum_e_ui+sum_i_ui)/total_population, label='UI', linestyle=(0,(5,5)), color='k', linewidth=4, alpha=1)
plt.plot(t_int,(sum_e_eg+sum_i_eg)/total_population, label='EG', linestyle=(0,(3,1,1,1)), color='tab:brown', linewidth=4, alpha=1)
plt.plot(t_int,(sum_e_hd+sum_i_hd)/total_population, label='HD', linestyle=(0,(3,5,1,5)), color='tab:red', linewidth=4, alpha=1)
plt.plot(t_int,(sum_e_sp+sum_i_sp)/total_population, label='SP', linestyle=(0,(5,1)), color='tab:orange', linewidth=4, alpha=1)
plt.plot(t_int,(sum_e_rw+sum_i_rw)/total_population, label='CF', linestyle='dashed', color='tab:green', linewidth=4, alpha=1)
plt.plot(t_int,(sum_e_lf50+sum_i_lf50)/total_population, label='LF(1/2)', linestyle=(0,(1,1)), color="tab:cyan", linewidth=4, alpha=1)
plt.plot(t_int,(sum_e_lf10+sum_i_lf10)/total_population, label='LF(1/10)', linestyle=(0,(3,1,1,1,1,1)), color='tab:purple', linewidth=4, alpha=1)
plt.plot(t_int,(sum_e_lf02+sum_i_lf02)/total_population, label='LF(1/50)', linestyle='dashdot', color='tab:blue', linewidth=4)
leg = plt.legend(fontsize=18, bbox_to_anchor=(.5, 1.4), ncol=3, loc='upper center', handlelength=4)
for i in leg.legendHandles:
    i.set_linewidth(4)
plt.xticks(size=18)
plt.yticks(size=18)
plt.xlabel('Day', fontsize=22)
plt.ylabel('Active Cases',fontsize=22)
plt.savefig("facebook_curves_cluster_85.tiff", bbox_inches='tight', format='tiff', dpi=400, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

### Simulation for interventions that start on day 0, cluster initialization

In [ ]:
reduced_weight = .1
target_perc = [.05, .1, .15, .2, .25, .3, .35, .4, .45, .5]
num_scenarios = len(target_perc)
num_trials = 50

maxCases_ni = np.zeros((num_trials,num_scenarios))
maxCases_ui = np.zeros((num_trials,num_scenarios))
maxCases_hd = np.zeros((num_trials,num_scenarios))
maxCases_eg = np.zeros((num_trials,num_scenarios))
maxCases_sp = np.zeros((num_trials,num_scenarios))
maxCases_rw = np.zeros((num_trials,num_scenarios))
maxCases_lf50 = np.zeros((num_trials,num_scenarios))
maxCases_lf10 = np.zeros((num_trials,num_scenarios))
maxCases_lf02 = np.zeros((num_trials,num_scenarios))
totalCases_ni = np.zeros((num_trials,num_scenarios))
totalCases_ui = np.zeros((num_trials,num_scenarios))
totalCases_hd = np.zeros((num_trials,num_scenarios))
totalCases_eg = np.zeros((num_trials,num_scenarios))
totalCases_sp = np.zeros((num_trials,num_scenarios))
totalCases_rw = np.zeros((num_trials,num_scenarios))
totalCases_lf50 = np.zeros((num_trials,num_scenarios))
totalCases_lf10 = np.zeros((num_trials,num_scenarios))
totalCases_lf02 = np.zeros((num_trials,num_scenarios))

for k in range(num_scenarios):
    
    perc = target_perc[k]
    A_ui = ((1-(1-reduced_weight)*perc)*nx.adjacency_matrix(G) + sp.sparse.eye(G.number_of_nodes())).tocsc()
    A_hd = create_weighted_adjacency_from_degree_dist(G, perc, weight=reduced_weight)
    A_eg = create_weighted_adjacency_from_node_betweenness(G, eigenvector_centrality, perc, weight=reduced_weight)
    A_sp = create_weighted_adjacency_from_edge_betweenness(G, shortest_path_betweenness, perc, weight=reduced_weight)
    A_rw = create_weighted_adjacency_from_edge_betweenness(G, current_flow_betweenness, perc, weight=reduced_weight)
    A_lf50 = create_weighted_adjacency_from_edge_betweenness(G, local50_betweenness, perc, weight=reduced_weight)
    A_lf10 = create_weighted_adjacency_from_edge_betweenness(G, local10_betweenness, perc, weight=reduced_weight)
    A_lf02 = create_weighted_adjacency_from_edge_betweenness(G, local02_betweenness, perc, weight=reduced_weight)
    
    for trial in range(num_trials):
        
        print(f'perc = {perc:.2f}, trial {trial+1:d} of {num_trials:d}', end="\n")
    
        S_0 = np.zeros(n)
        E_0 = np.zeros(n)
        I_0 = np.zeros(n)
        R_0 = np.zeros(n)
        for i in range(n):
            S_0[i] = populations[map_to_original[i+1]]
                
        # the following lines set up cluster initialization
        for i in initial_cluster:
            E_0[i] = 0
            I_0[i] = populations[map_to_original[i+1]] * .001
            S_0[i] = S_0[i] - E_0[i] - I_0[i]
        
        ini_cond = np.column_stack((S_0,E_0,I_0,R_0))
        t_span = (0.,1200.)
        
        sol_ni = Main.run_network_seir(Main.scipyCSC_to_julia(A), ini_cond, t_span, beta=beta)
        sol_ui = Main.run_network_seir(Main.scipyCSC_to_julia(A_ui), ini_cond, t_span, beta=beta)
        sol_hd = Main.run_network_seir(Main.scipyCSC_to_julia(A_hd), ini_cond, t_span, beta=beta)
        sol_eg = Main.run_network_seir(Main.scipyCSC_to_julia(A_eg), ini_cond, t_span, beta=beta)
        sol_sp = Main.run_network_seir(Main.scipyCSC_to_julia(A_sp), ini_cond, t_span, beta=beta)
        sol_rw = Main.run_network_seir(Main.scipyCSC_to_julia(A_rw), ini_cond, t_span, beta=beta)
        sol_lf50 = Main.run_network_seir(Main.scipyCSC_to_julia(A_lf50), ini_cond, t_span, beta=beta)
        sol_lf10 = Main.run_network_seir(Main.scipyCSC_to_julia(A_lf10), ini_cond, t_span, beta=beta)
        sol_lf02 = Main.run_network_seir(Main.scipyCSC_to_julia(A_lf02), ini_cond, t_span, beta=beta) 
        maxCases_ni[trial,k] = get_max_active_cases(sol_ni)
        maxCases_ui[trial,k] = get_max_active_cases(sol_ui)
        maxCases_hd[trial,k] = get_max_active_cases(sol_hd)
        maxCases_eg[trial,k] = get_max_active_cases(sol_eg)
        maxCases_sp[trial,k] = get_max_active_cases(sol_sp)
        maxCases_rw[trial,k] = get_max_active_cases(sol_rw)
        maxCases_lf50[trial,k] = get_max_active_cases(sol_lf50)
        maxCases_lf10[trial,k] = get_max_active_cases(sol_lf10)
        maxCases_lf02[trial,k] = get_max_active_cases(sol_lf02)
        totalCases_ni[trial,k] = get_total_active_cases(sol_ni)
        totalCases_ui[trial,k] = get_total_active_cases(sol_ui)
        totalCases_hd[trial,k] = get_total_active_cases(sol_hd)
        totalCases_eg[trial,k] = get_total_active_cases(sol_eg)
        totalCases_sp[trial,k] = get_total_active_cases(sol_sp)
        totalCases_rw[trial,k] = get_total_active_cases(sol_rw)
        totalCases_lf50[trial,k] = get_total_active_cases(sol_lf50)
        totalCases_lf10[trial,k] = get_total_active_cases(sol_lf10)
        totalCases_lf02[trial,k] = get_total_active_cases(sol_lf02)

In [ ]:
maxCases_ni_mean = np.mean(maxCases_ni/total_population, axis=0)
maxCases_ui_mean = np.mean(maxCases_ui/total_population, axis=0)
maxCases_hd_mean = np.mean(maxCases_hd/total_population, axis=0)
maxCases_eg_mean = np.mean(maxCases_eg/total_population, axis=0)
maxCases_sp_mean = np.mean(maxCases_sp/total_population, axis=0)
maxCases_rw_mean = np.mean(maxCases_rw/total_population, axis=0)
maxCases_lf50_mean = np.mean(maxCases_lf50/total_population, axis=0)
maxCases_lf10_mean = np.mean(maxCases_lf10/total_population, axis=0)
maxCases_lf02_mean = np.mean(maxCases_lf02/total_population, axis=0)
totalCases_ni_mean = np.mean(totalCases_ni/total_population, axis=0)
totalCases_ui_mean = np.mean(totalCases_ui/total_population, axis=0)
totalCases_hd_mean = np.mean(totalCases_hd/total_population, axis=0)
totalCases_eg_mean = np.mean(totalCases_eg/total_population, axis=0)
totalCases_sp_mean = np.mean(totalCases_sp/total_population, axis=0)
totalCases_rw_mean = np.mean(totalCases_rw/total_population, axis=0)
totalCases_lf50_mean = np.mean(totalCases_lf50/total_population, axis=0)
totalCases_lf10_mean = np.mean(totalCases_lf10/total_population, axis=0)
totalCases_lf02_mean = np.mean(totalCases_lf02/total_population, axis=0)

plt.figure(figsize=(9.3,4.5))
plt.plot(target_perc, maxCases_ni_mean, label='NI', linestyle='-', color='dimgray', linewidth=6)
plt.plot(target_perc, maxCases_ui_mean, label='UI', linestyle=(0,(5,5)), color='k', linewidth=6)
plt.plot(target_perc, maxCases_hd_mean, label='HD', linestyle=(0,(3,5,1,5)), color='tab:red', linewidth=6)
plt.plot(target_perc, maxCases_eg_mean, label='EG', linestyle=(0,(3,1,1,1)), color='tab:brown', linewidth=6)
plt.plot(target_perc, maxCases_sp_mean, label='SP', linestyle=(0,(5,1)), color='tab:orange', linewidth=6)
plt.plot(target_perc, maxCases_rw_mean, label='CF', linestyle='dashed', color='tab:green', linewidth=6)
plt.plot(target_perc, maxCases_lf50_mean, label='LF(1/2)', linestyle=(0,(1,1)), color="tab:cyan", linewidth=6)
plt.plot(target_perc, maxCases_lf10_mean, label='LF(1/10)', linestyle=(0,(3,1,1,1,1,1)), color='tab:purple', linewidth=6)
plt.plot(target_perc, maxCases_lf02_mean, label='LF(1/50)', linestyle='dashdot', color='tab:blue', linewidth=6)
leg = plt.legend(fontsize=18, bbox_to_anchor=(.5, 1.4), ncol=3, loc='upper center', handlelength=4)
for i in leg.legendHandles:
    i.set_linewidth(4)
plt.xticks((.1, .2, .3, .4, .5), ('$10\%$', '$20\%$', '$30\%$', '$40\%$', '$50\%$'), color='k', size=18)
plt.yticks(size=18)
plt.xlabel('Percentage of Targeted Edges', fontsize=22)
plt.ylabel('Epidemic Peak',fontsize=22)
plt.savefig("facebook_epipeak_cluster_85.tiff", bbox_inches='tight', format='tiff', dpi=400, pil_kwargs={"compression": "tiff_lzw"})
plt.show()
                                      
plt.figure(figsize=(9.3,4.5))
plt.plot(target_perc, totalCases_ni_mean, label='NI', linestyle='-', color='dimgray', linewidth=6)
plt.plot(target_perc, totalCases_ui_mean, label='UI', linestyle=(0,(5,5)), color='k', linewidth=6)
plt.plot(target_perc, totalCases_hd_mean, label='HD', linestyle=(0,(3,5,1,5)), color='tab:red', linewidth=6)
plt.plot(target_perc, totalCases_eg_mean, label='EG', linestyle=(0,(3,1,1,1)), color='tab:brown', linewidth=6)
plt.plot(target_perc, totalCases_sp_mean, label='SP', linestyle=(0,(5,1)), color='tab:orange', linewidth=6)
plt.plot(target_perc, totalCases_rw_mean, label='CF', linestyle='dashed', color='tab:green', linewidth=6)
plt.plot(target_perc, totalCases_lf50_mean, label='LF(1/2)', linestyle=(0,(1,1)), color="tab:cyan", linewidth=6)
plt.plot(target_perc, totalCases_lf10_mean, label='LF(1/10)', linestyle=(0,(3,1,1,1,1,1)), color='tab:purple', linewidth=6)
plt.plot(target_perc, totalCases_lf02_mean, label='LF(1/50)', linestyle='dashdot', color='tab:blue', linewidth=6)
leg = plt.legend(fontsize=18, bbox_to_anchor=(.5, 1.4), ncol=3, loc='upper center', handlelength=4)
for i in leg.legendHandles:
    i.set_linewidth(4)
plt.xticks((.1, .2, .3, .4, .5), ('$10\%$', '$20\%$', '$30\%$', '$40\%$', '$50\%$'), color='k', size=18)
plt.yticks(size=18)
plt.xlabel('Percentage of Targeted Edges', fontsize=22)
plt.ylabel('Final Epidemic Size',fontsize=22)
plt.savefig("facebook_episize_cluster_85.tiff", bbox_inches='tight', format='tiff', dpi=400, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

## Scenario 1c: 85% final size, random initialization + delay

In [ ]:
beta=0.0315

#### Verify that $\beta$ is set appropriately

In [ ]:
# initial conditions
S_0 = np.zeros(n)
E_0 = np.zeros(n)
I_0 = np.zeros(n)
R_0 = np.zeros(n)
for i in range(n):
    S_0[i] = populations[map_to_original[i+1]]

for i in sample(range(n),31):
    E_0[i] = 0
    I_0[i] = populations[map_to_original[i+1]] * .001
    S_0[i] = S_0[i] - E_0[i] - I_0[i]
    
ini_cond = np.column_stack((S_0,E_0,I_0,R_0))

# time span
t_end = 400.
t_span = (0.,t_end)

sol = Main.run_network_seir(Main.scipyCSC_to_julia(A), ini_cond, t_span, beta=beta)
print(f'This should be around 0.85: {get_total_active_cases(sol)/total_population:.6f}')

##### Sample epidemic curve: random initialization, delay = 50 days, 25% intervention

In [ ]:
# time span
t_delay = 50.
t_end = 200.
t_span = (t_delay, t_end)

# delayed initial conditions
sol = Main.run_network_seir(Main.scipyCSC_to_julia(A), ini_cond, (0.,t_delay), beta=beta) 
ini_cond_delay = sol[-1]

# simulation
sol_ni = Main.run_network_seir(Main.scipyCSC_to_julia(A), ini_cond_delay, t_span, beta=beta)
sol_ui = Main.run_network_seir(Main.scipyCSC_to_julia(A_ui), ini_cond_delay, t_span, beta=beta)
sol_hd = Main.run_network_seir(Main.scipyCSC_to_julia(A_hd), ini_cond_delay, t_span, beta=beta)
sol_eg = Main.run_network_seir(Main.scipyCSC_to_julia(A_eg), ini_cond_delay, t_span, beta=beta)
sol_sp = Main.run_network_seir(Main.scipyCSC_to_julia(A_sp), ini_cond_delay, t_span, beta=beta)
sol_rw = Main.run_network_seir(Main.scipyCSC_to_julia(A_rw), ini_cond_delay, t_span, beta=beta)
sol_lf50 = Main.run_network_seir(Main.scipyCSC_to_julia(A_lf50), ini_cond_delay, t_span, beta=beta)
sol_lf10 = Main.run_network_seir(Main.scipyCSC_to_julia(A_lf10), ini_cond_delay, t_span, beta=beta)
sol_lf02 = Main.run_network_seir(Main.scipyCSC_to_julia(A_lf02), ini_cond_delay, t_span, beta=beta)

# get curves and make plot
sum_s, sum_e, sum_i, sum_r = get_data_for_plotting(sol)
sum_s_ni, sum_e_ni, sum_i_ni, sum_r_ni = get_data_for_plotting(sol_ni)
sum_s_ui, sum_e_ui, sum_i_ui, sum_r_ui = get_data_for_plotting(sol_ui)
sum_s_hd, sum_e_hd, sum_i_hd, sum_r_hd = get_data_for_plotting(sol_hd)
sum_s_eg, sum_e_eg, sum_i_eg, sum_r_eg = get_data_for_plotting(sol_eg)
sum_s_sp, sum_e_sp, sum_i_sp, sum_r_sp = get_data_for_plotting(sol_sp)
sum_s_rw, sum_e_rw, sum_i_rw, sum_r_rw = get_data_for_plotting(sol_rw)
sum_s_lf50, sum_e_lf50, sum_i_lf50, sum_r_lf50 = get_data_for_plotting(sol_lf50)
sum_s_lf10, sum_e_lf10, sum_i_lf10, sum_r_lf10 = get_data_for_plotting(sol_lf10)
sum_s_lf02, sum_e_lf02, sum_i_lf02, sum_r_lf02 = get_data_for_plotting(sol_lf02)

t_ini = np.linspace(0, int(t_delay), num=int(t_delay)+1)
t_int = np.linspace(t_delay, int(t_end), num=int(t_end-t_delay)+1)

plt.figure(figsize=(9.3,4.5))
plt.plot(t_ini,(sum_e+sum_i)/total_population, linestyle='-', color='dimgray', linewidth=4, alpha=1)
plt.plot(t_int,(sum_e_ni+sum_i_ni)/total_population, label='NI', linestyle='-', color='dimgray', linewidth=4, alpha=1)
plt.plot(t_int,(sum_e_ui+sum_i_ui)/total_population, label='UI', linestyle=(0,(5,5)), color='k', linewidth=4, alpha=1)
plt.plot(t_int,(sum_e_eg+sum_i_eg)/total_population, label='EG', linestyle=(0,(3,1,1,1)), color='tab:brown', linewidth=4)
plt.plot(t_int,(sum_e_hd+sum_i_hd)/total_population, label='HD', linestyle=(0,(3,5,1,5)), color='tab:red', linewidth=4, alpha=1)
plt.plot(t_int,(sum_e_sp+sum_i_sp)/total_population, label='SP', linestyle=(0,(5,1)), color='tab:orange', linewidth=4, alpha=1)
plt.plot(t_int,(sum_e_rw+sum_i_rw)/total_population, label='CF', linestyle='dashed', color='tab:green', linewidth=4, alpha=1)
plt.plot(t_int,(sum_e_lf50+sum_i_lf50)/total_population, label='LF(1/2)', linestyle=(0,(1,1)), color="tab:cyan", linewidth=4, alpha=1)
plt.plot(t_int,(sum_e_lf10+sum_i_lf10)/total_population, label='LF(1/10)', linestyle=(0,(3,1,1,1,1,1)), color='tab:purple', linewidth=4)
plt.plot(t_int,(sum_e_lf02+sum_i_lf02)/total_population, label='LF(1/50)', linestyle='dashdot', color='tab:blue', linewidth=4)
leg = plt.legend(fontsize=18, bbox_to_anchor=(.5, 1.4), ncol=3, loc='upper center', handlelength=4)
for i in leg.legendHandles:
    i.set_linewidth(4)
plt.xticks(size=18)
plt.yticks((.0, .1), size=18)
plt.xlabel('Day', fontsize=22)
plt.ylabel('Active Cases',fontsize=22)
plt.savefig("facebook_curves_randinit_85_delay50.tiff", bbox_inches='tight', format='tiff', dpi=400, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

## Simulation for interventions that start on day 50, random initialization

In [ ]:
reduced_weight = .1
target_perc = [.05, .1, .15, .2, .25, .3, .35, .4, .45, .5]
num_scenarios = len(target_perc)
num_trials = 50

maxCases_ni = np.zeros((num_trials,num_scenarios))
maxCases_ui = np.zeros((num_trials,num_scenarios))
maxCases_hd = np.zeros((num_trials,num_scenarios))
maxCases_eg = np.zeros((num_trials,num_scenarios))
maxCases_sp = np.zeros((num_trials,num_scenarios))
maxCases_rw = np.zeros((num_trials,num_scenarios))
maxCases_lf50 = np.zeros((num_trials,num_scenarios))
maxCases_lf10 = np.zeros((num_trials,num_scenarios))
maxCases_lf02 = np.zeros((num_trials,num_scenarios))
totalCases_ni = np.zeros((num_trials,num_scenarios))
totalCases_ui = np.zeros((num_trials,num_scenarios))
totalCases_hd = np.zeros((num_trials,num_scenarios))
totalCases_eg = np.zeros((num_trials,num_scenarios))
totalCases_sp = np.zeros((num_trials,num_scenarios))
totalCases_rw = np.zeros((num_trials,num_scenarios))
totalCases_lf50 = np.zeros((num_trials,num_scenarios))
totalCases_lf10 = np.zeros((num_trials,num_scenarios))
totalCases_lf02 = np.zeros((num_trials,num_scenarios))

for k in range(num_scenarios):
    
    perc = target_perc[k]
    A_ui = ((1-(1-reduced_weight)*perc)*nx.adjacency_matrix(G) + sp.sparse.eye(G.number_of_nodes())).tocsc()
    A_hd = create_weighted_adjacency_from_degree_dist(G, perc, weight=reduced_weight)
    A_eg = create_weighted_adjacency_from_node_betweenness(G, eigenvector_centrality, perc, weight=reduced_weight)
    A_sp = create_weighted_adjacency_from_edge_betweenness(G, shortest_path_betweenness, perc, weight=reduced_weight)
    A_rw = create_weighted_adjacency_from_edge_betweenness(G, current_flow_betweenness, perc, weight=reduced_weight)
    A_lf50 = create_weighted_adjacency_from_edge_betweenness(G, local50_betweenness, perc, weight=reduced_weight)
    A_lf10 = create_weighted_adjacency_from_edge_betweenness(G, local10_betweenness, perc, weight=reduced_weight)
    A_lf02 = create_weighted_adjacency_from_edge_betweenness(G, local02_betweenness, perc, weight=reduced_weight)
    
    for trial in range(num_trials):
        
        print(f'perc = {perc:.2f}, trial {trial+1:d} of {num_trials}', end="\n")
    
        S_0 = np.zeros(n)
        E_0 = np.zeros(n)
        I_0 = np.zeros(n)
        R_0 = np.zeros(n)
        for i in range(n):
            S_0[i] = populations[map_to_original[i+1]]
        
        for i in sample(range(n),31):
            E_0[i] = 0
            I_0[i] = populations[map_to_original[i+1]] * .001
            S_0[i] = S_0[i] - E_0[i] - I_0[i]
        
        ini_cond = np.column_stack((S_0,E_0,I_0,R_0))
        t_span = (0.,50.)
        sol = Main.run_network_seir(Main.scipyCSC_to_julia(A), ini_cond, t_span, beta=beta)
        
        ini_cond = sol[-1]
        t_span = (0.,500.)
        
        sol_ni = Main.run_network_seir(Main.scipyCSC_to_julia(A), ini_cond, t_span, beta=beta)
        sol_ui = Main.run_network_seir(Main.scipyCSC_to_julia(A_ui), ini_cond, t_span, beta=beta)
        sol_hd = Main.run_network_seir(Main.scipyCSC_to_julia(A_hd), ini_cond, t_span, beta=beta)
        sol_eg = Main.run_network_seir(Main.scipyCSC_to_julia(A_eg), ini_cond, t_span, beta=beta)
        sol_sp = Main.run_network_seir(Main.scipyCSC_to_julia(A_sp), ini_cond, t_span, beta=beta)
        sol_rw = Main.run_network_seir(Main.scipyCSC_to_julia(A_rw), ini_cond, t_span, beta=beta)
        sol_lf50 = Main.run_network_seir(Main.scipyCSC_to_julia(A_lf50), ini_cond, t_span, beta=beta)
        sol_lf10 = Main.run_network_seir(Main.scipyCSC_to_julia(A_lf10), ini_cond, t_span, beta=beta)
        sol_lf02 = Main.run_network_seir(Main.scipyCSC_to_julia(A_lf02), ini_cond, t_span, beta=beta) 
        maxCases_ni[trial,k] = get_max_active_cases(sol_ni)
        maxCases_ui[trial,k] = get_max_active_cases(sol_ui)
        maxCases_hd[trial,k] = get_max_active_cases(sol_hd)
        maxCases_eg[trial,k] = get_max_active_cases(sol_eg)
        maxCases_sp[trial,k] = get_max_active_cases(sol_sp)
        maxCases_rw[trial,k] = get_max_active_cases(sol_rw)
        maxCases_lf50[trial,k] = get_max_active_cases(sol_lf50)
        maxCases_lf10[trial,k] = get_max_active_cases(sol_lf10)
        maxCases_lf02[trial,k] = get_max_active_cases(sol_lf02)
        totalCases_ni[trial,k] = get_total_active_cases(sol_ni)
        totalCases_ui[trial,k] = get_total_active_cases(sol_ui)
        totalCases_hd[trial,k] = get_total_active_cases(sol_hd)
        totalCases_eg[trial,k] = get_total_active_cases(sol_eg)
        totalCases_sp[trial,k] = get_total_active_cases(sol_sp)
        totalCases_rw[trial,k] = get_total_active_cases(sol_rw)
        totalCases_lf50[trial,k] = get_total_active_cases(sol_lf50)
        totalCases_lf10[trial,k] = get_total_active_cases(sol_lf10)
        totalCases_lf02[trial,k] = get_total_active_cases(sol_lf02)

In [ ]:
maxCases_ni_mean = np.mean(maxCases_ni/total_population, axis=0)
maxCases_ui_mean = np.mean(maxCases_ui/total_population, axis=0)
maxCases_hd_mean = np.mean(maxCases_hd/total_population, axis=0)
maxCases_eg_mean = np.mean(maxCases_eg/total_population, axis=0)
maxCases_sp_mean = np.mean(maxCases_sp/total_population, axis=0)
maxCases_rw_mean = np.mean(maxCases_rw/total_population, axis=0)
maxCases_lf50_mean = np.mean(maxCases_lf50/total_population, axis=0)
maxCases_lf10_mean = np.mean(maxCases_lf10/total_population, axis=0)
maxCases_lf02_mean = np.mean(maxCases_lf02/total_population, axis=0)
totalCases_ni_mean = np.mean(totalCases_ni/total_population, axis=0)
totalCases_ui_mean = np.mean(totalCases_ui/total_population, axis=0)
totalCases_hd_mean = np.mean(totalCases_hd/total_population, axis=0)
totalCases_eg_mean = np.mean(totalCases_eg/total_population, axis=0)
totalCases_sp_mean = np.mean(totalCases_sp/total_population, axis=0)
totalCases_rw_mean = np.mean(totalCases_rw/total_population, axis=0)
totalCases_lf50_mean = np.mean(totalCases_lf50/total_population, axis=0)
totalCases_lf10_mean = np.mean(totalCases_lf10/total_population, axis=0)
totalCases_lf02_mean = np.mean(totalCases_lf02/total_population, axis=0)

plt.figure(figsize=(9.3,4.5))
plt.plot(target_perc, maxCases_ni_mean, label='NI', linestyle='-', color='dimgray', linewidth=6)
plt.plot(target_perc, maxCases_ui_mean, label='UI', linestyle=(0,(5,5)), color='k', linewidth=6)
plt.plot(target_perc, maxCases_hd_mean, label='HD', linestyle=(0,(3,5,1,5)), color='tab:red', linewidth=6)
plt.plot(target_perc, maxCases_eg_mean, label='EG', linestyle=(0,(3,1,1,1)), color='tab:brown', linewidth=6)
plt.plot(target_perc, maxCases_sp_mean, label='SP', linestyle=(0,(5,1)), color='tab:orange', linewidth=6)
plt.plot(target_perc, maxCases_rw_mean, label='CF', linestyle='dashed', color='tab:green', linewidth=6)
plt.plot(target_perc, maxCases_lf50_mean, label='LF(1/2)', linestyle=(0,(1,1)), color="tab:cyan", linewidth=6)
plt.plot(target_perc, maxCases_lf10_mean, label='LF(1/10)', linestyle=(0,(3,1,1,1,1,1)), color='tab:purple', linewidth=6)
plt.plot(target_perc, maxCases_lf02_mean, label='LF(1/50)', linestyle='dashdot', color='tab:blue', linewidth=6)
leg = plt.legend(fontsize=18, bbox_to_anchor=(.5, 1.4), ncol=3, loc='upper center', handlelength=4)
for i in leg.legendHandles:
    i.set_linewidth(4)
plt.xticks((.1, .2, .3, .4, .5), ('$10\%$', '$20\%$', '$30\%$', '$40\%$', '$50\%$'), color='k', size=18)
plt.yticks((.1, .2), size=18)
plt.xlabel('Percentage of Targeted Edges', fontsize=22)
plt.ylabel('Epidemic Peak',fontsize=22)
plt.savefig("facebook_epipeak_randinit_85_delay50.tiff", bbox_inches='tight', format='tiff', dpi=400, pil_kwargs={"compression": "tiff_lzw"})
plt.show()
                                      
plt.figure(figsize=(9.3,4.5))
plt.plot(target_perc, totalCases_ni_mean, label='NI', linestyle='-', color='dimgray', linewidth=6)
plt.plot(target_perc, totalCases_ui_mean, label='UI', linestyle=(0,(5,5)), color='k', linewidth=6)
plt.plot(target_perc, totalCases_hd_mean, label='HD', linestyle=(0,(3,5,1,5)), color='tab:red', linewidth=6)
plt.plot(target_perc, totalCases_eg_mean, label='EG', linestyle=(0,(3,1,1,1)), color='tab:brown', linewidth=6)
plt.plot(target_perc, totalCases_sp_mean, label='SP', linestyle=(0,(5,1)), color='tab:orange', linewidth=6)
plt.plot(target_perc, totalCases_rw_mean, label='CF', linestyle='dashed', color='tab:green', linewidth=6)
plt.plot(target_perc, totalCases_lf50_mean, label='LF(1/2)', linestyle=(0,(1,1)), color="tab:cyan", linewidth=6)
plt.plot(target_perc, totalCases_lf10_mean, label='LF(1/10)', linestyle=(0,(3,1,1,1,1,1)), color='tab:purple', linewidth=6)
plt.plot(target_perc, totalCases_lf02_mean, label='LF(1/50)', linestyle='dashdot', color='tab:blue', linewidth=6)
leg = plt.legend(fontsize=18, bbox_to_anchor=(.5, 1.4), ncol=3, loc='upper center', handlelength=4)
for i in leg.legendHandles:
    i.set_linewidth(4)
plt.xticks((.1, .2, .3, .4, .5), ('$10\%$', '$20\%$', '$30\%$', '$40\%$', '$50\%$'), color='k', size=18)
plt.yticks(size=18)
plt.xlabel('Percentage of Targeted Edges', fontsize=22)
plt.ylabel('Final Epidemic Size',fontsize=22)
plt.savefig("facebook_episize_randinit_85_delay50.tiff", bbox_inches='tight', format='tiff', dpi=400, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

## Scenario 2: 70% final size, random initialization

In [ ]:
beta=0.0236

#### Verify that $\beta$ is set appropriately

In [ ]:
# initial conditions
S_0 = np.zeros(n)
E_0 = np.zeros(n)
I_0 = np.zeros(n)
R_0 = np.zeros(n)
for i in range(n):
    S_0[i] = populations[map_to_original[i+1]] 
    
# the following lines set up random initializatiom
for i in sample(range(n),31):
    E_0[i] = 0
    I_0[i] = populations[map_to_original[i+1]] * .001
    S_0[i] = S_0[i] - E_0[i] - I_0[i]
    
ini_cond = np.column_stack((S_0,E_0,I_0,R_0))

# time span
t_end = 400.
t_span = (0.,t_end)

sol = Main.run_network_seir(Main.scipyCSC_to_julia(A), ini_cond, t_span, beta=beta)
print(f'This should be around 0.85: {get_total_active_cases(sol)/total_population:.6f}')

##### Sample epidemic curve: random initialization,  25% intervention, no delay

In [ ]:
perc = .25
reduced_weight = .1

In [ ]:
A_ui = ((1-(1-reduced_weight)*perc)*nx.adjacency_matrix(G) + sp.sparse.eye(G.number_of_nodes())).tocsc()
A_hd = create_weighted_adjacency_from_degree_dist(G, perc, weight=reduced_weight)
A_eg = create_weighted_adjacency_from_node_betweenness(G, eigenvector_centrality, perc, weight=reduced_weight)
A_sp = create_weighted_adjacency_from_edge_betweenness(G, shortest_path_betweenness, perc, weight=reduced_weight)
A_rw = create_weighted_adjacency_from_edge_betweenness(G, current_flow_betweenness, perc, weight=reduced_weight)
A_lf50 = create_weighted_adjacency_from_edge_betweenness(G, local50_betweenness, perc, weight=reduced_weight)
A_lf10 = create_weighted_adjacency_from_edge_betweenness(G, local10_betweenness, perc, weight=reduced_weight)
A_lf02 = create_weighted_adjacency_from_edge_betweenness(G, local02_betweenness, perc, weight=reduced_weight)

In [ ]:
# time span
t_end = 400.
t_span = (0., t_end)

# initial conditions
n = G.number_of_nodes()
S_0 = np.zeros(n)
E_0 = np.zeros(n)
I_0 = np.zeros(n)
R_0 = np.zeros(n)

for i in range(n):
    S_0[i] = populations[map_to_original[i+1]]

for i in sample(range(n),31):
    E_0[i] = 0
    I_0[i] = populations[map_to_original[i+1]] * .001
    S_0[i] = S_0[i] - E_0[i] - I_0[i]

ini_cond = np.column_stack((S_0,E_0,I_0,R_0))

# simulation
sol_ni = Main.run_network_seir(Main.scipyCSC_to_julia(A), ini_cond, t_span, beta=beta)
sol_ui = Main.run_network_seir(Main.scipyCSC_to_julia(A_ui), ini_cond, t_span, beta=beta)
sol_hd = Main.run_network_seir(Main.scipyCSC_to_julia(A_hd), ini_cond, t_span, beta=beta)
sol_eg = Main.run_network_seir(Main.scipyCSC_to_julia(A_eg), ini_cond, t_span, beta=beta)
sol_sp = Main.run_network_seir(Main.scipyCSC_to_julia(A_sp), ini_cond, t_span, beta=beta)
sol_rw = Main.run_network_seir(Main.scipyCSC_to_julia(A_rw), ini_cond, t_span, beta=beta)
sol_lf50 = Main.run_network_seir(Main.scipyCSC_to_julia(A_lf50), ini_cond, t_span, beta=beta)
sol_lf10 = Main.run_network_seir(Main.scipyCSC_to_julia(A_lf10), ini_cond, t_span, beta=beta)
sol_lf02 = Main.run_network_seir(Main.scipyCSC_to_julia(A_lf02), ini_cond, t_span, beta=beta)

# get curves and make plot
sum_s_ni, sum_e_ni, sum_i_ni, sum_r_ni = get_data_for_plotting(sol_ni)
sum_s_ui, sum_e_ui, sum_i_ui, sum_r_ui = get_data_for_plotting(sol_ui)
sum_s_hd, sum_e_hd, sum_i_hd, sum_r_hd = get_data_for_plotting(sol_hd)
sum_s_eg, sum_e_eg, sum_i_eg, sum_r_eg = get_data_for_plotting(sol_eg)
sum_s_sp, sum_e_sp, sum_i_sp, sum_r_sp = get_data_for_plotting(sol_sp)
sum_s_rw, sum_e_rw, sum_i_rw, sum_r_rw = get_data_for_plotting(sol_rw)
sum_s_lf50, sum_e_lf50, sum_i_lf50, sum_r_lf50 = get_data_for_plotting(sol_lf50)
sum_s_lf10, sum_e_lf10, sum_i_lf10, sum_r_lf10 = get_data_for_plotting(sol_lf10)
sum_s_lf02, sum_e_lf02, sum_i_lf02, sum_r_lf02 = get_data_for_plotting(sol_lf02)

t_int = np.linspace(0, int(t_end), num=int(t_end)+1)

plt.figure(figsize=(9.3,4.5))
plt.plot(t_int,(sum_e_ni+sum_i_ni)/total_population, label='NI', linestyle='-', color='dimgray', linewidth=3, alpha=1)
plt.plot(t_int,(sum_e_ui+sum_i_ui)/total_population, label='UI', linestyle=(0,(5,5)), color='k', linewidth=4, alpha=1)
plt.plot(t_int,(sum_e_eg+sum_i_eg)/total_population, label='EG', linestyle=(0,(3,1,1,1)), color='tab:brown', linewidth=4, alpha=1)
plt.plot(t_int,(sum_e_hd+sum_i_hd)/total_population, label='HD', linestyle=(0,(3,5,1,5)), color='tab:red', linewidth=4, alpha=1)
plt.plot(t_int,(sum_e_sp+sum_i_sp)/total_population, label='SP', linestyle=(0,(5,1)), color='tab:orange', linewidth=4, alpha=1)
plt.plot(t_int,(sum_e_rw+sum_i_rw)/total_population, label='CF', linestyle='dashed', color='tab:green', linewidth=4, alpha=1)
plt.plot(t_int,(sum_e_lf50+sum_i_lf50)/total_population, label='LF(1/2)', linestyle=(0,(1,1)), color="tab:cyan", linewidth=4, alpha=1)
plt.plot(t_int,(sum_e_lf10+sum_i_lf10)/total_population, label='LF(1/10)', linestyle=(0,(3,1,1,1,1,1)), color='tab:purple', linewidth=4, alpha=1)
plt.plot(t_int,(sum_e_lf02+sum_i_lf02)/total_population, label='LF(1/50)', linestyle='dashdot', color='tab:blue', linewidth=4)
leg = plt.legend(fontsize=18, bbox_to_anchor=(.5, 1.4), ncol=3, loc='upper center', handlelength=4)
for i in leg.legendHandles:
    i.set_linewidth(4)
plt.xticks(size=18)
plt.yticks((.0, .1), size=18)
plt.xlabel('Day', fontsize=22)
plt.ylabel('Active Cases',fontsize=22)
plt.savefig("facebook_curves_randinit_70.tiff", bbox_inches='tight', format='tiff', dpi=400, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

### Simulation for interventions that start on day 0, random initialization

In [ ]:
reduced_weight = .1
target_perc = [.05, .1, .15, .2, .25, .3, .35, .4, .45, .5]
num_scenarios = len(target_perc)
num_trials = 50

maxCases_ni = np.zeros((num_trials,num_scenarios))
maxCases_ui = np.zeros((num_trials,num_scenarios))
maxCases_hd = np.zeros((num_trials,num_scenarios))
maxCases_eg = np.zeros((num_trials,num_scenarios))
maxCases_sp = np.zeros((num_trials,num_scenarios))
maxCases_rw = np.zeros((num_trials,num_scenarios))
maxCases_lf50 = np.zeros((num_trials,num_scenarios))
maxCases_lf10 = np.zeros((num_trials,num_scenarios))
maxCases_lf02 = np.zeros((num_trials,num_scenarios))
totalCases_ni = np.zeros((num_trials,num_scenarios))
totalCases_ui = np.zeros((num_trials,num_scenarios))
totalCases_hd = np.zeros((num_trials,num_scenarios))
totalCases_eg = np.zeros((num_trials,num_scenarios))
totalCases_sp = np.zeros((num_trials,num_scenarios))
totalCases_rw = np.zeros((num_trials,num_scenarios))
totalCases_lf50 = np.zeros((num_trials,num_scenarios))
totalCases_lf10 = np.zeros((num_trials,num_scenarios))
totalCases_lf02 = np.zeros((num_trials,num_scenarios))

for k in range(num_scenarios):
    
    perc = target_perc[k]
    A_ui = ((1-(1-reduced_weight)*perc)*nx.adjacency_matrix(G) + sp.sparse.eye(G.number_of_nodes())).tocsc()
    A_hd = create_weighted_adjacency_from_degree_dist(G, perc, weight=reduced_weight)
    A_eg = create_weighted_adjacency_from_node_betweenness(G, eigenvector_centrality, perc, weight=reduced_weight)
    A_sp = create_weighted_adjacency_from_edge_betweenness(G, shortest_path_betweenness, perc, weight=reduced_weight)
    A_rw = create_weighted_adjacency_from_edge_betweenness(G, current_flow_betweenness, perc, weight=reduced_weight)
    A_lf50 = create_weighted_adjacency_from_edge_betweenness(G, local50_betweenness, perc, weight=reduced_weight)
    A_lf10 = create_weighted_adjacency_from_edge_betweenness(G, local10_betweenness, perc, weight=reduced_weight)
    A_lf02 = create_weighted_adjacency_from_edge_betweenness(G, local02_betweenness, perc, weight=reduced_weight)
    
    for trial in range(num_trials):
        
        print(f'perc = {perc:.2f}, trial {trial+1:d} of {num_trials:d}', end="\n")
    
        S_0 = np.zeros(n)
        E_0 = np.zeros(n)
        I_0 = np.zeros(n)
        R_0 = np.zeros(n)
        for i in range(n):
            S_0[i] = populations[map_to_original[i+1]]

        for i in sample(range(n),31):
            E_0[i] = 0
            I_0[i] = populations[map_to_original[i+1]] * .001
            S_0[i] = S_0[i] - E_0[i] - I_0[i]

        ini_cond = np.column_stack((S_0,E_0,I_0,R_0))
        t_span = (0.,1200.)
        
        sol_ni = Main.run_network_seir(Main.scipyCSC_to_julia(A), ini_cond, t_span, beta=beta)
        sol_ui = Main.run_network_seir(Main.scipyCSC_to_julia(A_ui), ini_cond, t_span, beta=beta)
        sol_hd = Main.run_network_seir(Main.scipyCSC_to_julia(A_hd), ini_cond, t_span, beta=beta)
        sol_eg = Main.run_network_seir(Main.scipyCSC_to_julia(A_eg), ini_cond, t_span, beta=beta)
        sol_sp = Main.run_network_seir(Main.scipyCSC_to_julia(A_sp), ini_cond, t_span, beta=beta)
        sol_rw = Main.run_network_seir(Main.scipyCSC_to_julia(A_rw), ini_cond, t_span, beta=beta)
        sol_lf50 = Main.run_network_seir(Main.scipyCSC_to_julia(A_lf50), ini_cond, t_span, beta=beta)
        sol_lf10 = Main.run_network_seir(Main.scipyCSC_to_julia(A_lf10), ini_cond, t_span, beta=beta)
        sol_lf02 = Main.run_network_seir(Main.scipyCSC_to_julia(A_lf02), ini_cond, t_span, beta=beta) 
        maxCases_ni[trial,k] = get_max_active_cases(sol_ni)
        maxCases_ui[trial,k] = get_max_active_cases(sol_ui)
        maxCases_hd[trial,k] = get_max_active_cases(sol_hd)
        maxCases_eg[trial,k] = get_max_active_cases(sol_eg)
        maxCases_sp[trial,k] = get_max_active_cases(sol_sp)
        maxCases_rw[trial,k] = get_max_active_cases(sol_rw)
        maxCases_lf50[trial,k] = get_max_active_cases(sol_lf50)
        maxCases_lf10[trial,k] = get_max_active_cases(sol_lf10)
        maxCases_lf02[trial,k] = get_max_active_cases(sol_lf02)
        totalCases_ni[trial,k] = get_total_active_cases(sol_ni)
        totalCases_ui[trial,k] = get_total_active_cases(sol_ui)
        totalCases_hd[trial,k] = get_total_active_cases(sol_hd)
        totalCases_eg[trial,k] = get_total_active_cases(sol_eg)
        totalCases_sp[trial,k] = get_total_active_cases(sol_sp)
        totalCases_rw[trial,k] = get_total_active_cases(sol_rw)
        totalCases_lf50[trial,k] = get_total_active_cases(sol_lf50)
        totalCases_lf10[trial,k] = get_total_active_cases(sol_lf10)
        totalCases_lf02[trial,k] = get_total_active_cases(sol_lf02)

In [ ]:
maxCases_ni_mean = np.mean(maxCases_ni/total_population, axis=0)
maxCases_ui_mean = np.mean(maxCases_ui/total_population, axis=0)
maxCases_hd_mean = np.mean(maxCases_hd/total_population, axis=0)
maxCases_eg_mean = np.mean(maxCases_eg/total_population, axis=0)
maxCases_sp_mean = np.mean(maxCases_sp/total_population, axis=0)
maxCases_rw_mean = np.mean(maxCases_rw/total_population, axis=0)
maxCases_lf50_mean = np.mean(maxCases_lf50/total_population, axis=0)
maxCases_lf10_mean = np.mean(maxCases_lf10/total_population, axis=0)
maxCases_lf02_mean = np.mean(maxCases_lf02/total_population, axis=0)
totalCases_ni_mean = np.mean(totalCases_ni/total_population, axis=0)
totalCases_ui_mean = np.mean(totalCases_ui/total_population, axis=0)
totalCases_hd_mean = np.mean(totalCases_hd/total_population, axis=0)
totalCases_eg_mean = np.mean(totalCases_eg/total_population, axis=0)
totalCases_sp_mean = np.mean(totalCases_sp/total_population, axis=0)
totalCases_rw_mean = np.mean(totalCases_rw/total_population, axis=0)
totalCases_lf50_mean = np.mean(totalCases_lf50/total_population, axis=0)
totalCases_lf10_mean = np.mean(totalCases_lf10/total_population, axis=0)
totalCases_lf02_mean = np.mean(totalCases_lf02/total_population, axis=0)

plt.figure(figsize=(9.3,4.5))
plt.plot(target_perc, maxCases_ni_mean, label='NI', linestyle='-', color='dimgray', linewidth=6)
plt.plot(target_perc, maxCases_ui_mean, label='UI', linestyle=(0,(5,5)), color='k', linewidth=6)
plt.plot(target_perc, maxCases_hd_mean, label='HD', linestyle=(0,(3,5,1,5)), color='tab:red', linewidth=6)
plt.plot(target_perc, maxCases_eg_mean, label='EG', linestyle=(0,(3,1,1,1)), color='tab:brown', linewidth=6)
plt.plot(target_perc, maxCases_sp_mean, label='SP', linestyle=(0,(5,1)), color='tab:orange', linewidth=6)
plt.plot(target_perc, maxCases_rw_mean, label='CF', linestyle='dashed', color='tab:green', linewidth=6)
plt.plot(target_perc, maxCases_lf50_mean, label='LF(1/2)', linestyle=(0,(1,1)), color="tab:cyan", linewidth=6)
plt.plot(target_perc, maxCases_lf10_mean, label='LF(1/10)', linestyle=(0,(3,1,1,1,1,1)), color='tab:purple', linewidth=6)
plt.plot(target_perc, maxCases_lf02_mean, label='LF(1/50)', linestyle='dashdot', color='tab:blue', linewidth=6)
leg = plt.legend(fontsize=18, bbox_to_anchor=(.5, 1.4), ncol=3, loc='upper center', handlelength=4)
for i in leg.legendHandles:
    i.set_linewidth(4)
plt.xticks((.1, .2, .3, .4, .5), ('$10\%$', '$20\%$', '$30\%$', '$40\%$', '$50\%$'), color='k', size=18)
plt.yticks((.0, .1), size=18)
plt.xlabel('Percentage of Targeted Edges', fontsize=22)
plt.ylabel('Epidemic Peak',fontsize=22)
plt.savefig("facebook_epipeak_randinit_70.tiff", bbox_inches='tight', format='tiff', dpi=400, pil_kwargs={"compression": "tiff_lzw"})
plt.show()
                                      
plt.figure(figsize=(9.3,4.5))
plt.plot(target_perc, totalCases_ni_mean, label='NI', linestyle='-', color='dimgray', linewidth=6)
plt.plot(target_perc, totalCases_ui_mean, label='UI', linestyle=(0,(5,5)), color='k', linewidth=6)
plt.plot(target_perc, totalCases_hd_mean, label='HD', linestyle=(0,(3,5,1,5)), color='tab:red', linewidth=6)
plt.plot(target_perc, totalCases_eg_mean, label='EG', linestyle=(0,(3,1,1,1)), color='tab:brown', linewidth=6)
plt.plot(target_perc, totalCases_sp_mean, label='SP', linestyle=(0,(5,1)), color='tab:orange', linewidth=6)
plt.plot(target_perc, totalCases_rw_mean, label='CF', linestyle='dashed', color='tab:green', linewidth=6)
plt.plot(target_perc, totalCases_lf50_mean, label='LF(1/2)', linestyle=(0,(1,1)), color="tab:cyan", linewidth=6)
plt.plot(target_perc, totalCases_lf10_mean, label='LF(1/10)', linestyle=(0,(3,1,1,1,1,1)), color='tab:purple', linewidth=6)
plt.plot(target_perc, totalCases_lf02_mean, label='LF(1/50)', linestyle='dashdot', color='tab:blue', linewidth=6)
leg = plt.legend(fontsize=18, bbox_to_anchor=(.5, 1.4), ncol=3, loc='upper center', handlelength=4)
for i in leg.legendHandles:
    i.set_linewidth(4)
plt.xticks((.1, .2, .3, .4, .5), ('$10\%$', '$20\%$', '$30\%$', '$40\%$', '$50\%$'), color='k', size=18)
plt.yticks(size=18)
plt.xlabel('Percentage of Targeted Edges', fontsize=22)
plt.ylabel('Final Epidemic Size',fontsize=22)
plt.savefig("facebook_episize_randinit_70.tiff", bbox_inches='tight', format='tiff', dpi=400, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

## Scenario 3: $\beta$ is set to give 55% final size

In [ ]:
beta=0.0195

#### Verify that $\beta$ is set appropriately

In [ ]:
# initial conditions
S_0 = np.zeros(n)
E_0 = np.zeros(n)
I_0 = np.zeros(n)
R_0 = np.zeros(n)
for i in range(n):
    S_0[i] = populations[map_to_original[i+1]] 
    
# the following lines set up random initializatiom
for i in sample(range(n),31):
    E_0[i] = 0
    I_0[i] = populations[map_to_original[i+1]] * .001
    S_0[i] = S_0[i] - E_0[i] - I_0[i]
    
ini_cond = np.column_stack((S_0,E_0,I_0,R_0))

# time span
t_end = 400.
t_span = (0.,t_end)

sol = Main.run_network_seir(Main.scipyCSC_to_julia(A), ini_cond, t_span, beta=beta)
print(f'This should be around 0.50: {get_total_active_cases(sol)/total_population:.6f}')

##### Sample epidemic curve: random initialization, 25% intervention, no delay

In [ ]:
perc = .25
reduced_weight = .1

In [ ]:
A_ui = ((1-(1-reduced_weight)*perc)*nx.adjacency_matrix(G) + sp.sparse.eye(G.number_of_nodes())).tocsc()
A_hd = create_weighted_adjacency_from_degree_dist(G, perc, weight=reduced_weight)
A_eg = create_weighted_adjacency_from_node_betweenness(G, eigenvector_centrality, perc, weight=reduced_weight)
A_sp = create_weighted_adjacency_from_edge_betweenness(G, shortest_path_betweenness, perc, weight=reduced_weight)
A_rw = create_weighted_adjacency_from_edge_betweenness(G, current_flow_betweenness, perc, weight=reduced_weight)
A_lf50 = create_weighted_adjacency_from_edge_betweenness(G, local50_betweenness, perc, weight=reduced_weight)
A_lf10 = create_weighted_adjacency_from_edge_betweenness(G, local10_betweenness, perc, weight=reduced_weight)
A_lf02 = create_weighted_adjacency_from_edge_betweenness(G, local02_betweenness, perc, weight=reduced_weight)

In [ ]:
# time span
t_end = 500.
t_span = (0., t_end)

# initial conditions
n = G.number_of_nodes()
S_0 = np.zeros(n)
E_0 = np.zeros(n)
I_0 = np.zeros(n)
R_0 = np.zeros(n)

for i in range(n):
    S_0[i] = populations[map_to_original[i+1]]

for i in sample(range(n),31):
    E_0[i] = 0
    I_0[i] = populations[map_to_original[i+1]] * .001
    S_0[i] = S_0[i] - E_0[i] - I_0[i]

ini_cond = np.column_stack((S_0,E_0,I_0,R_0))

# simulation
sol_ni = Main.run_network_seir(Main.scipyCSC_to_julia(A), ini_cond, t_span, beta=beta)
sol_ui = Main.run_network_seir(Main.scipyCSC_to_julia(A_ui), ini_cond, t_span, beta=beta)
sol_hd = Main.run_network_seir(Main.scipyCSC_to_julia(A_hd), ini_cond, t_span, beta=beta)
sol_eg = Main.run_network_seir(Main.scipyCSC_to_julia(A_eg), ini_cond, t_span, beta=beta)
sol_sp = Main.run_network_seir(Main.scipyCSC_to_julia(A_sp), ini_cond, t_span, beta=beta)
sol_rw = Main.run_network_seir(Main.scipyCSC_to_julia(A_rw), ini_cond, t_span, beta=beta)
sol_lf50 = Main.run_network_seir(Main.scipyCSC_to_julia(A_lf50), ini_cond, t_span, beta=beta)
sol_lf10 = Main.run_network_seir(Main.scipyCSC_to_julia(A_lf10), ini_cond, t_span, beta=beta)
sol_lf02 = Main.run_network_seir(Main.scipyCSC_to_julia(A_lf02), ini_cond, t_span, beta=beta)

# get curves and make plot
sum_s_ni, sum_e_ni, sum_i_ni, sum_r_ni = get_data_for_plotting(sol_ni)
sum_s_ui, sum_e_ui, sum_i_ui, sum_r_ui = get_data_for_plotting(sol_ui)
sum_s_hd, sum_e_hd, sum_i_hd, sum_r_hd = get_data_for_plotting(sol_hd)
sum_s_eg, sum_e_eg, sum_i_eg, sum_r_eg = get_data_for_plotting(sol_eg)
sum_s_sp, sum_e_sp, sum_i_sp, sum_r_sp = get_data_for_plotting(sol_sp)
sum_s_rw, sum_e_rw, sum_i_rw, sum_r_rw = get_data_for_plotting(sol_rw)
sum_s_lf50, sum_e_lf50, sum_i_lf50, sum_r_lf50 = get_data_for_plotting(sol_lf50)
sum_s_lf10, sum_e_lf10, sum_i_lf10, sum_r_lf10 = get_data_for_plotting(sol_lf10)
sum_s_lf02, sum_e_lf02, sum_i_lf02, sum_r_lf02 = get_data_for_plotting(sol_lf02)

t_int = np.linspace(0, int(t_end), num=int(t_end)+1)

plt.figure(figsize=(9.3,4.5))
plt.plot(t_int,(sum_e_ni+sum_i_ni)/total_population, label='NI', linestyle='-', color='dimgray', linewidth=3, alpha=1)
plt.plot(t_int,(sum_e_ui+sum_i_ui)/total_population, label='UI', linestyle=(0,(5,5)), color='k', linewidth=4, alpha=1)
plt.plot(t_int,(sum_e_eg+sum_i_eg)/total_population, label='EG', linestyle=(0,(3,1,1,1)), color='tab:brown', linewidth=4, alpha=1)
plt.plot(t_int,(sum_e_hd+sum_i_hd)/total_population, label='HD', linestyle=(0,(3,5,1,5)), color='tab:red', linewidth=4, alpha=1)
plt.plot(t_int,(sum_e_sp+sum_i_sp)/total_population, label='SP', linestyle=(0,(5,1)), color='tab:orange', linewidth=4, alpha=1)
plt.plot(t_int,(sum_e_rw+sum_i_rw)/total_population, label='CF', linestyle='dashed', color='tab:green', linewidth=4, alpha=1)
plt.plot(t_int,(sum_e_lf50+sum_i_lf50)/total_population, label='LF(1/2)', linestyle=(0,(1,1)), color="tab:cyan", linewidth=4, alpha=1)
plt.plot(t_int,(sum_e_lf10+sum_i_lf10)/total_population, label='LF(1/10)', linestyle=(0,(3,1,1,1,1,1)), color='tab:purple', linewidth=4, alpha=1)
plt.plot(t_int,(sum_e_lf02+sum_i_lf02)/total_population, label='LF(1/50)', linestyle='dashdot', color='tab:blue', linewidth=4)
leg = plt.legend(fontsize=18, bbox_to_anchor=(.5, 1.4), ncol=3, loc='upper center', handlelength=4)
for i in leg.legendHandles:
    i.set_linewidth(4)
plt.xticks(size=18)
plt.yticks(size=18)
plt.xlabel('Day', fontsize=22)
plt.ylabel('Active Cases',fontsize=22)
plt.savefig("facebook_curves_randinit_55.tiff", bbox_inches='tight', format='tiff', dpi=400, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

### Simulation for interventions that start on day 0, random initialization

In [ ]:
reduced_weight = .1
target_perc = [.05, .1, .15, .2, .25, .3, .35, .4, .45, .5]
num_scenarios = len(target_perc)
num_trials = 50

maxCases_ni = np.zeros((num_trials,num_scenarios))
maxCases_ui = np.zeros((num_trials,num_scenarios))
maxCases_hd = np.zeros((num_trials,num_scenarios))
maxCases_eg = np.zeros((num_trials,num_scenarios))
maxCases_sp = np.zeros((num_trials,num_scenarios))
maxCases_rw = np.zeros((num_trials,num_scenarios))
maxCases_lf50 = np.zeros((num_trials,num_scenarios))
maxCases_lf10 = np.zeros((num_trials,num_scenarios))
maxCases_lf02 = np.zeros((num_trials,num_scenarios))
totalCases_ni = np.zeros((num_trials,num_scenarios))
totalCases_ui = np.zeros((num_trials,num_scenarios))
totalCases_hd = np.zeros((num_trials,num_scenarios))
totalCases_eg = np.zeros((num_trials,num_scenarios))
totalCases_sp = np.zeros((num_trials,num_scenarios))
totalCases_rw = np.zeros((num_trials,num_scenarios))
totalCases_lf50 = np.zeros((num_trials,num_scenarios))
totalCases_lf10 = np.zeros((num_trials,num_scenarios))
totalCases_lf02 = np.zeros((num_trials,num_scenarios))

for k in range(num_scenarios):
    
    perc = target_perc[k]
    A_ui = ((1-(1-reduced_weight)*perc)*nx.adjacency_matrix(G) + sp.sparse.eye(G.number_of_nodes())).tocsc()
    A_hd = create_weighted_adjacency_from_degree_dist(G, perc, weight=reduced_weight)
    A_eg = create_weighted_adjacency_from_node_betweenness(G, eigenvector_centrality, perc, weight=reduced_weight)
    A_sp = create_weighted_adjacency_from_edge_betweenness(G, shortest_path_betweenness, perc, weight=reduced_weight)
    A_rw = create_weighted_adjacency_from_edge_betweenness(G, current_flow_betweenness, perc, weight=reduced_weight)
    A_lf50 = create_weighted_adjacency_from_edge_betweenness(G, local50_betweenness, perc, weight=reduced_weight)
    A_lf10 = create_weighted_adjacency_from_edge_betweenness(G, local10_betweenness, perc, weight=reduced_weight)
    A_lf02 = create_weighted_adjacency_from_edge_betweenness(G, local02_betweenness, perc, weight=reduced_weight)
    
    for trial in range(num_trials):
        
        print(f'perc = {perc:.2f}, trial {trial+1:d} of {num_trials:d}', end="\n")
    
        S_0 = np.zeros(n)
        E_0 = np.zeros(n)
        I_0 = np.zeros(n)
        R_0 = np.zeros(n)
        for i in range(n):
            S_0[i] = populations[map_to_original[i+1]]

        for i in sample(range(n),31):
            E_0[i] = 0
            I_0[i] = populations[map_to_original[i+1]] * .001
            S_0[i] = S_0[i] - E_0[i] - I_0[i]

        ini_cond = np.column_stack((S_0,E_0,I_0,R_0))
        t_span = (0.,1500.)
        
        sol_ni = Main.run_network_seir(Main.scipyCSC_to_julia(A), ini_cond, t_span, beta=beta)
        sol_ui = Main.run_network_seir(Main.scipyCSC_to_julia(A_ui), ini_cond, t_span, beta=beta)
        sol_hd = Main.run_network_seir(Main.scipyCSC_to_julia(A_hd), ini_cond, t_span, beta=beta)
        sol_eg = Main.run_network_seir(Main.scipyCSC_to_julia(A_eg), ini_cond, t_span, beta=beta)
        sol_sp = Main.run_network_seir(Main.scipyCSC_to_julia(A_sp), ini_cond, t_span, beta=beta)
        sol_rw = Main.run_network_seir(Main.scipyCSC_to_julia(A_rw), ini_cond, t_span, beta=beta)
        sol_lf50 = Main.run_network_seir(Main.scipyCSC_to_julia(A_lf50), ini_cond, t_span, beta=beta)
        sol_lf10 = Main.run_network_seir(Main.scipyCSC_to_julia(A_lf10), ini_cond, t_span, beta=beta)
        sol_lf02 = Main.run_network_seir(Main.scipyCSC_to_julia(A_lf02), ini_cond, t_span, beta=beta) 
        maxCases_ni[trial,k] = get_max_active_cases(sol_ni)
        maxCases_ui[trial,k] = get_max_active_cases(sol_ui)
        maxCases_hd[trial,k] = get_max_active_cases(sol_hd)
        maxCases_eg[trial,k] = get_max_active_cases(sol_eg)
        maxCases_sp[trial,k] = get_max_active_cases(sol_sp)
        maxCases_rw[trial,k] = get_max_active_cases(sol_rw)
        maxCases_lf50[trial,k] = get_max_active_cases(sol_lf50)
        maxCases_lf10[trial,k] = get_max_active_cases(sol_lf10)
        maxCases_lf02[trial,k] = get_max_active_cases(sol_lf02)
        totalCases_ni[trial,k] = get_total_active_cases(sol_ni)
        totalCases_ui[trial,k] = get_total_active_cases(sol_ui)
        totalCases_hd[trial,k] = get_total_active_cases(sol_hd)
        totalCases_eg[trial,k] = get_total_active_cases(sol_eg)
        totalCases_sp[trial,k] = get_total_active_cases(sol_sp)
        totalCases_rw[trial,k] = get_total_active_cases(sol_rw)
        totalCases_lf50[trial,k] = get_total_active_cases(sol_lf50)
        totalCases_lf10[trial,k] = get_total_active_cases(sol_lf10)
        totalCases_lf02[trial,k] = get_total_active_cases(sol_lf02)

In [ ]:
maxCases_ni_mean = np.mean(maxCases_ni/total_population, axis=0)
maxCases_ui_mean = np.mean(maxCases_ui/total_population, axis=0)
maxCases_hd_mean = np.mean(maxCases_hd/total_population, axis=0)
maxCases_eg_mean = np.mean(maxCases_eg/total_population, axis=0)
maxCases_sp_mean = np.mean(maxCases_sp/total_population, axis=0)
maxCases_rw_mean = np.mean(maxCases_rw/total_population, axis=0)
maxCases_lf50_mean = np.mean(maxCases_lf50/total_population, axis=0)
maxCases_lf10_mean = np.mean(maxCases_lf10/total_population, axis=0)
maxCases_lf02_mean = np.mean(maxCases_lf02/total_population, axis=0)
totalCases_ni_mean = np.mean(totalCases_ni/total_population, axis=0)
totalCases_ui_mean = np.mean(totalCases_ui/total_population, axis=0)
totalCases_hd_mean = np.mean(totalCases_hd/total_population, axis=0)
totalCases_eg_mean = np.mean(totalCases_eg/total_population, axis=0)
totalCases_sp_mean = np.mean(totalCases_sp/total_population, axis=0)
totalCases_rw_mean = np.mean(totalCases_rw/total_population, axis=0)
totalCases_lf50_mean = np.mean(totalCases_lf50/total_population, axis=0)
totalCases_lf10_mean = np.mean(totalCases_lf10/total_population, axis=0)
totalCases_lf02_mean = np.mean(totalCases_lf02/total_population, axis=0)

plt.figure(figsize=(9.3,4.5))
plt.plot(target_perc, maxCases_ni_mean, label='NI', linestyle='-', color='dimgray', linewidth=6)
plt.plot(target_perc, maxCases_ui_mean, label='UI', linestyle=(0,(5,5)), color='k', linewidth=6)
plt.plot(target_perc, maxCases_hd_mean, label='HD', linestyle=(0,(3,5,1,5)), color='tab:red', linewidth=6)
plt.plot(target_perc, maxCases_eg_mean, label='EG', linestyle=(0,(3,1,1,1)), color='tab:brown', linewidth=6)
plt.plot(target_perc, maxCases_sp_mean, label='SP', linestyle=(0,(5,1)), color='tab:orange', linewidth=6)
plt.plot(target_perc, maxCases_rw_mean, label='CF', linestyle='dashed', color='tab:green', linewidth=6)
plt.plot(target_perc, maxCases_lf50_mean, label='LF(1/2)', linestyle=(0,(1,1)), color="tab:cyan", linewidth=6)
plt.plot(target_perc, maxCases_lf10_mean, label='LF(1/10)', linestyle=(0,(3,1,1,1,1,1)), color='tab:purple', linewidth=6)
plt.plot(target_perc, maxCases_lf02_mean, label='LF(1/50)', linestyle='dashdot', color='tab:blue', linewidth=6)
leg = plt.legend(fontsize=18, bbox_to_anchor=(.5, 1.4), ncol=3, loc='upper center', handlelength=4)
for i in leg.legendHandles:
    i.set_linewidth(4)
plt.xticks((.1, .2, .3, .4, .5), ('$10\%$', '$20\%$', '$30\%$', '$40\%$', '$50\%$'), color='k', size=18)
plt.yticks(size=18)
plt.xlabel('Percentage of Targeted Edges', fontsize=22)
plt.ylabel('Epidemic Peak',fontsize=22)
plt.savefig("facebook_epipeak_randinit_55.tiff", bbox_inches='tight', format='tiff', dpi=400, pil_kwargs={"compression": "tiff_lzw"})
plt.show()
                                      
plt.figure(figsize=(9.3,4.5))
plt.plot(target_perc, totalCases_ni_mean, label='NI', linestyle='-', color='dimgray', linewidth=6)
plt.plot(target_perc, totalCases_ui_mean, label='UI', linestyle=(0,(5,5)), color='k', linewidth=6)
plt.plot(target_perc, totalCases_hd_mean, label='HD', linestyle=(0,(3,5,1,5)), color='tab:red', linewidth=6)
plt.plot(target_perc, totalCases_eg_mean, label='EG', linestyle=(0,(3,1,1,1)), color='tab:brown', linewidth=6)
plt.plot(target_perc, totalCases_sp_mean, label='SP', linestyle=(0,(5,1)), color='tab:orange', linewidth=6)
plt.plot(target_perc, totalCases_rw_mean, label='CF', linestyle='dashed', color='tab:green', linewidth=6)
plt.plot(target_perc, totalCases_lf50_mean, label='LF(1/2)', linestyle=(0,(1,1)), color="tab:cyan", linewidth=6)
plt.plot(target_perc, totalCases_lf10_mean, label='LF(1/10)', linestyle=(0,(3,1,1,1,1,1)), color='tab:purple', linewidth=6)
plt.plot(target_perc, totalCases_lf02_mean, label='LF(1/50)', linestyle='dashdot', color='tab:blue', linewidth=6)
leg = plt.legend(fontsize=18, bbox_to_anchor=(.5, 1.4), ncol=3, loc='upper center', handlelength=4)
for i in leg.legendHandles:
    i.set_linewidth(4)
plt.xticks((.1, .2, .3, .4, .5), ('$10\%$', '$20\%$', '$30\%$', '$40\%$', '$50\%$'), color='k', size=18)
plt.yticks(size=18)
plt.xlabel('Percentage of Targeted Edges', fontsize=22)
plt.ylabel('Final Epidemic Size',fontsize=22)
plt.savefig("facebook_episize_randinit_55.tiff", bbox_inches='tight', format='tiff', dpi=400, pil_kwargs={"compression": "tiff_lzw"})
plt.show()

# Node immunization

#### Verify that $\beta$ is set appropriately

In [ ]:
# initial conditions
S_0 = np.zeros(n)
E_0 = np.zeros(n)
I_0 = np.zeros(n)
R_0 = np.zeros(n)
for i in range(n):
    S_0[i] = populations[map_to_original[i+1]]

for i in sample(range(n),31):
    E_0[i] = 0
    I_0[i] = populations[map_to_original[i+1]] * .001
    S_0[i] = S_0[i] - E_0[i] - I_0[i]
    
ini_cond = np.column_stack((S_0,E_0,I_0,R_0))

# time span
t_end = 400.
t_span = (0.,t_end)

sol = Main.run_network_seir(Main.scipyCSC_to_julia(A), ini_cond, t_span, beta=beta)
print(f'This should be around 0.85: {get_total_active_cases(sol)/total_population:.6f}')

In [ ]:
node_sp = edge_to_node_betweenness(shortest_path_betweenness, G.nodes())
node_rw = edge_to_node_betweenness(current_flow_betweenness, G.nodes())
node_lf50 = edge_to_node_betweenness(local50_betweenness, G.nodes())
node_lf10 = edge_to_node_betweenness(local10_betweenness, G.nodes())
node_lf02 = edge_to_node_betweenness(local02_betweenness, G.nodes())

In [ ]:
reduced_weight = 0
target_perc = [.05, .1, .15, .2, .25, .3]
num_scenarios = len(target_perc)
num_trials = 50

maxCases_ni = np.zeros((num_trials,num_scenarios))
maxCases_ui = np.zeros((num_trials,num_scenarios))
maxCases_hd = np.zeros((num_trials,num_scenarios))
maxCases_eg = np.zeros((num_trials,num_scenarios))
maxCases_sp = np.zeros((num_trials,num_scenarios))
maxCases_rw = np.zeros((num_trials,num_scenarios))
maxCases_lf50 = np.zeros((num_trials,num_scenarios))
maxCases_lf10 = np.zeros((num_trials,num_scenarios))
maxCases_lf02 = np.zeros((num_trials,num_scenarios))
totalCases_ni = np.zeros((num_trials,num_scenarios))
totalCases_ui = np.zeros((num_trials,num_scenarios))
totalCases_hd = np.zeros((num_trials,num_scenarios))
totalCases_eg = np.zeros((num_trials,num_scenarios))
totalCases_sp = np.zeros((num_trials,num_scenarios))
totalCases_rw = np.zeros((num_trials,num_scenarios))
totalCases_lf50 = np.zeros((num_trials,num_scenarios))
totalCases_lf10 = np.zeros((num_trials,num_scenarios))
totalCases_lf02 = np.zeros((num_trials,num_scenarios))

for k in range(num_scenarios):
    
    perc = target_perc[k]
    A_ui = create_weighted_adjacency_target_random_nodes(G, perc, weight=reduced_weight)
    A_hd = create_weighted_adjacency_from_degree_dist_target_nodes(G, perc, weight=reduced_weight)
    A_eg = create_weighted_adjacency_from_node_betweenness_target_nodes(G, eigenvector_centrality, perc, weight=reduced_weight)
    A_sp = create_weighted_adjacency_from_node_betweenness_target_nodes(G, node_sp, perc, weight=reduced_weight)
    A_rw = create_weighted_adjacency_from_node_betweenness_target_nodes(G, node_rw, perc, weight=reduced_weight)
    A_lf50 = create_weighted_adjacency_from_node_betweenness_target_nodes(G, node_lf50, perc, weight=reduced_weight)
    A_lf10 = create_weighted_adjacency_from_node_betweenness_target_nodes(G, node_lf10, perc, weight=reduced_weight)
    A_lf02 = create_weighted_adjacency_from_node_betweenness_target_nodes(G, node_lf02, perc, weight=reduced_weight)
    
    for trial in range(num_trials):
        
        print(f'perc = {perc:.2f}, trial {trial+1:d} of {num_trials:d}', end="\n")
    
        S_0 = np.zeros(n)
        E_0 = np.zeros(n)
        I_0 = np.zeros(n)
        R_0 = np.zeros(n)
        for i in range(n):
            S_0[i] = populations[map_to_original[i+1]]
        for i in sample(range(n),31):
            E_0[i] = 0
            I_0[i] = populations[map_to_original[i+1]] * .001
            S_0[i] = S_0[i] - E_0[i] - I_0[i]

        ini_cond = np.column_stack((S_0,E_0,I_0,R_0))
        t_span = (0.,1000.)
        
        sol_ni = Main.run_network_seir(Main.scipyCSC_to_julia(A), ini_cond, t_span, beta=beta)
        sol_ui = Main.run_network_seir(Main.scipyCSC_to_julia(A_ui), ini_cond, t_span, beta=beta)
        sol_hd = Main.run_network_seir(Main.scipyCSC_to_julia(A_hd), ini_cond, t_span, beta=beta)
        sol_eg = Main.run_network_seir(Main.scipyCSC_to_julia(A_eg), ini_cond, t_span, beta=beta)
        sol_sp = Main.run_network_seir(Main.scipyCSC_to_julia(A_sp), ini_cond, t_span, beta=beta)
        sol_rw = Main.run_network_seir(Main.scipyCSC_to_julia(A_rw), ini_cond, t_span, beta=beta)
        sol_lf50 = Main.run_network_seir(Main.scipyCSC_to_julia(A_lf50), ini_cond, t_span, beta=beta)
        sol_lf10 = Main.run_network_seir(Main.scipyCSC_to_julia(A_lf10), ini_cond, t_span, beta=beta)
        sol_lf02 = Main.run_network_seir(Main.scipyCSC_to_julia(A_lf02), ini_cond, t_span, beta=beta) 
        maxCases_ni[trial,k] = get_max_active_cases(sol_ni)
        maxCases_ui[trial,k] = get_max_active_cases(sol_ui)
        maxCases_hd[trial,k] = get_max_active_cases(sol_hd)
        maxCases_eg[trial,k] = get_max_active_cases(sol_eg)
        maxCases_sp[trial,k] = get_max_active_cases(sol_sp)
        maxCases_rw[trial,k] = get_max_active_cases(sol_rw)
        maxCases_lf50[trial,k] = get_max_active_cases(sol_lf50)
        maxCases_lf10[trial,k] = get_max_active_cases(sol_lf10)
        maxCases_lf02[trial,k] = get_max_active_cases(sol_lf02)
        totalCases_ni[trial,k] = get_total_active_cases(sol_ni)
        totalCases_ui[trial,k] = get_total_active_cases(sol_ui)
        totalCases_hd[trial,k] = get_total_active_cases(sol_hd)
        totalCases_eg[trial,k] = get_total_active_cases(sol_eg)
        totalCases_sp[trial,k] = get_total_active_cases(sol_sp)
        totalCases_rw[trial,k] = get_total_active_cases(sol_rw)
        totalCases_lf50[trial,k] = get_total_active_cases(sol_lf50)
        totalCases_lf10[trial,k] = get_total_active_cases(sol_lf10)
        totalCases_lf02[trial,k] = get_total_active_cases(sol_lf02)

In [ ]:
maxCases_ni_mean = np.mean(maxCases_ni/total_population, axis=0)
maxCases_ui_mean = np.mean(maxCases_ui/total_population, axis=0)
maxCases_hd_mean = np.mean(maxCases_hd/total_population, axis=0)
maxCases_eg_mean = np.mean(maxCases_eg/total_population, axis=0)
maxCases_sp_mean = np.mean(maxCases_sp/total_population, axis=0)
maxCases_rw_mean = np.mean(maxCases_rw/total_population, axis=0)
maxCases_lf50_mean = np.mean(maxCases_lf50/total_population, axis=0)
maxCases_lf10_mean = np.mean(maxCases_lf10/total_population, axis=0)
maxCases_lf02_mean = np.mean(maxCases_lf02/total_population, axis=0)
totalCases_ni_mean = np.mean(totalCases_ni/total_population, axis=0)
totalCases_ui_mean = np.mean(totalCases_ui/total_population, axis=0)
totalCases_hd_mean = np.mean(totalCases_hd/total_population, axis=0)
totalCases_eg_mean = np.mean(totalCases_eg/total_population, axis=0)
totalCases_sp_mean = np.mean(totalCases_sp/total_population, axis=0)
totalCases_rw_mean = np.mean(totalCases_rw/total_population, axis=0)
totalCases_lf50_mean = np.mean(totalCases_lf50/total_population, axis=0)
totalCases_lf10_mean = np.mean(totalCases_lf10/total_population, axis=0)
totalCases_lf02_mean = np.mean(totalCases_lf02/total_population, axis=0)

plt.figure(figsize=(9.3,4.5))
plt.plot(target_perc, maxCases_ni_mean, label='NI', linestyle='-', color='dimgray', linewidth=6)
plt.plot(target_perc, maxCases_ui_mean, label='UI', linestyle=(0,(5,5)), color='k', linewidth=6)
plt.plot(target_perc, maxCases_hd_mean, label='HD', linestyle=(0,(3,5,1,5)), color='tab:red', linewidth=6)
plt.plot(target_perc, maxCases_eg_mean, label='EG', linestyle=(0,(3,1,1,1)), color='tab:brown', linewidth=6)
plt.plot(target_perc, maxCases_sp_mean, label='SP', linestyle=(0,(5,1)), color='tab:orange', linewidth=6)
plt.plot(target_perc, maxCases_rw_mean, label='CF', linestyle='dashed', color='tab:green', linewidth=6)
plt.plot(target_perc, maxCases_lf50_mean, label='LF(1/2)', linestyle=(0,(1,1)), color="tab:cyan", linewidth=6)
plt.plot(target_perc, maxCases_lf10_mean, label='LF(1/10)', linestyle=(0,(3,1,1,1,1,1)), color='tab:purple', linewidth=6)
plt.plot(target_perc, maxCases_lf02_mean, label='LF(1/50)', linestyle='dashdot', color='tab:blue', linewidth=6)
leg = plt.legend(fontsize=18, bbox_to_anchor=(.5, 1.4), ncol=3, loc='upper center', handlelength=4)
for i in leg.legendHandles:
    i.set_linewidth(4)
plt.xticks((.05, .1, .15, .2, .25, .3), ('$5\%$', '$10\%$', '$15\%$', '$20\%$', '$25\%$', '$30\%$'), color='k', size=18)
plt.yticks( size=18)
plt.xlabel('Percentage of Targeted Nodes', fontsize=22)
plt.ylabel('Epidemic Peak',fontsize=22)
plt.savefig("facebook_epipeak_randinit_85_immu.png", bbox_inches='tight', format='png', dpi=400)
plt.show()
                                      
plt.figure(figsize=(9.3,4.5))
plt.plot(target_perc, totalCases_ni_mean, label='NI', linestyle='-', color='dimgray', linewidth=6)
plt.plot(target_perc, totalCases_ui_mean, label='UI', linestyle=(0,(5,5)), color='k', linewidth=6)
plt.plot(target_perc, totalCases_hd_mean, label='HD', linestyle=(0,(3,5,1,5)), color='tab:red', linewidth=6)
plt.plot(target_perc, totalCases_eg_mean, label='EG', linestyle=(0,(3,1,1,1)), color='tab:brown', linewidth=6)
plt.plot(target_perc, totalCases_sp_mean, label='SP', linestyle=(0,(5,1)), color='tab:orange', linewidth=6)
plt.plot(target_perc, totalCases_rw_mean, label='CF', linestyle='dashed', color='tab:green', linewidth=6)
plt.plot(target_perc, totalCases_lf50_mean, label='LF(1/2)', linestyle=(0,(1,1)), color="tab:cyan", linewidth=6)
plt.plot(target_perc, totalCases_lf10_mean, label='LF(1/10)', linestyle=(0,(3,1,1,1,1,1)), color='tab:purple', linewidth=6)
plt.plot(target_perc, totalCases_lf02_mean, label='LF(1/50)', linestyle='dashdot', color='tab:blue', linewidth=6)
leg = plt.legend(fontsize=18, bbox_to_anchor=(.5, 1.4), ncol=3, loc='upper center', handlelength=4)
for i in leg.legendHandles:
    i.set_linewidth(4)
plt.xticks((.05, .1, .15, .2, .25, .3), ('$5\%$', '$10\%$', '$15\%$', '$20\%$', '$25\%$', '$30\%$'), color='k', size=18)
plt.yticks(size=18)
plt.xlabel('Percentage of Targeted Nodes', fontsize=22)
plt.ylabel('Final Epidemic Size',fontsize=22)
plt.savefig("facebook_episize_randinit_85_immu.png", bbox_inches='tight', format='png', dpi=400)
plt.show()